In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

In [124]:
import xxhash
h = xxhash.xxh64()

In [12]:
import os

In [3]:
# import numexpr as ne

In [4]:
from itertools import starmap, product, combinations, chain, permutations

In [5]:
from funcy import *

In [6]:
from functools import reduce

In [7]:
from tqdm import tqdm

from joblib import Parallel, delayed, Memory

J = -1
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def par(gen_expr, j=None, backend=None, verbose=None, prefer=None):
    if j is None:
        j = J
    if backend is None:
        backend = BACKEND
    if verbose is None:
        verbose = V
    if prefer is None:
        prefer = PREFER
    return Parallel(n_jobs=j, backend=backend, verbose=verbose, prefer=prefer)(gen_expr)

def identity(x):
    return x

In [8]:
from random import choice

In [9]:
CAREFUL = False

In [2]:
import numpy as np
myint = np.int8

In [10]:
import sparse

In [11]:
from scipy.special import binom#, comb

In [13]:
import torch

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    
    print(torch.cuda.get_device_name(0))
    total_mem_MB = torch.cuda.get_device_properties(device).total_memory / 1e6
    print('Total Memory: {0}'.format(total_mem_MB) )
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2070
Total Memory: 8367.439872
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [15]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

gpu_int8_ttype = torch.cuda.CharTensor
gpu_int16_ttype = torch.cuda.ShortTensor

my_ttype = gpu_int8_ttype
my_dtype = torch.uint8

def t(ndarray):
    if ndarray.dtype == myint:
        return torch.tensor(ndarray.astype(np.int16)).type(my_ttype)
    return torch.tensor(ndarray).type(my_ttype)

# Read in (or make) object vectors

## Make

In [16]:
m = 5

In [17]:
max_num_objects = 2 ** m
max_num_objects

max_num_partial_fvs = (2 + 1) ** m
max_num_partial_fvs

32

243

In [18]:
def make_random_pfv():
    return np.random.randint(3, size=m, dtype=myint) - 1

In [19]:
max_num_objects
actual_num_objects = np.random.randint(max_num_objects)
# actual_num_objects = 40
actual_num_objects

assert actual_num_objects < max_num_objects

32

24

In [20]:
def zeroToMinusOne(u):
    return np.array([x if x == 1 else -1 for x in u])

def makeRandomObjects(l, num_features, as_ndarray=False):
    l = actual_num_objects
    m = num_features
    objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
    objects = tuple(map(np.array, objects))
    objects = tuple([zeroToMinusOne(o) for o in objects])
    if not as_ndarray:
        return objects
    return np.array(objects)

# objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
# objects = tuple(map(np.array, objects))
# objects = tuple([zeroToMinusOne(o) for o in objects])
objects = makeRandomObjects(actual_num_objects, m)
l = len(objects)



actual_num_objects = len(objects)
actual_num_objects
objects

18

(array([ 1,  1,  1, -1, -1]),
 array([-1, -1, -1, -1, -1]),
 array([-1, -1,  1,  1, -1]),
 array([ 1,  1, -1, -1, -1]),
 array([ 1, -1, -1, -1,  1]),
 array([-1, -1,  1, -1,  1]),
 array([ 1,  1,  1, -1,  1]),
 array([ 1,  1, -1,  1,  1]),
 array([ 1, -1, -1,  1,  1]),
 array([ 1, -1,  1, -1,  1]),
 array([-1,  1, -1, -1,  1]),
 array([ 1, -1,  1,  1, -1]),
 array([ 1, -1, -1, -1, -1]),
 array([-1, -1,  1,  1,  1]),
 array([ 1, -1,  1, -1, -1]),
 array([1, 1, 1, 1, 1]),
 array([-1,  1,  1,  1,  1]),
 array([ 1, -1,  1,  1,  1]))

In [21]:
objectMap = np.array(objects) #np.array([objects[i] for i in range(l)])
objectMap.shape
objectMap
objectMap[0]

O = objectMap

(18, 5)

array([[ 1,  1,  1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1,  1,  1, -1],
       [ 1,  1, -1, -1, -1],
       [ 1, -1, -1, -1,  1],
       [-1, -1,  1, -1,  1],
       [ 1,  1,  1, -1,  1],
       [ 1,  1, -1,  1,  1],
       [ 1, -1, -1,  1,  1],
       [ 1, -1,  1, -1,  1],
       [-1,  1, -1, -1,  1],
       [ 1, -1,  1,  1, -1],
       [ 1, -1, -1, -1, -1],
       [-1, -1,  1,  1,  1],
       [ 1, -1,  1, -1, -1],
       [ 1,  1,  1,  1,  1],
       [-1,  1,  1,  1,  1],
       [ 1, -1,  1,  1,  1]])

array([ 1,  1,  1, -1, -1])

## Read-in

In [22]:
%ls *.npy

brh.npy  hayes.npy


In [23]:
objectMap = np.load('brh.npy')
objectMap.shape

l, m = objectMap.shape
actual_num_objects = l

O = objectMap
objects = tuple(objectMap)

(91, 23)

In [114]:
str(objects[0])

'[-1 -1 -1 -1 -1 -1 -1  1  1  0  0  0  0  0  0  0 -1 -1 -1  1 -1  0  0]'

In [115]:
objects
sorted(objects, key=lambda o: str(o))

(array([-1, -1, -1, -1, -1, -1, -1,  1,  1,  0,  0,  0,  0,  0,  0,  0, -1,
        -1, -1,  1, -1,  0,  0], dtype=int8),
 array([-1,  1,  1, -1, -1,  1,  1, -1,  0,  0,  0,  0,  1, -1, -1,  1, -1,
         1, -1, -1, -1,  0,  0], dtype=int8),
 array([-1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  0,  0,  0,  0, -1,
        -1, -1, -1, -1,  0,  0], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1, -1,  1,
        -1, -1, -1, -1,  1,  1], dtype=int8),
 array([-1,  1, -1, -1, -1,  1,  1,  1,  1,  0,  0,  0, -1,  1,  1, -1, -1,
        -1, -1, -1, -1,  0, -1], dtype=int8),
 array([-1,  1,  1, -1, -1,  1,  1, -1,  0,  0,  0,  0,  1, -1,  1, -1, -1,
         1, -1, -1, -1,  0,  0], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1,  1,  1, -1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1,  1, -1,  1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([-1,  1,  1, -1, 

[array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1,  1, -1,  1,
        -1, -1, -1, -1,  1,  1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
        -1, -1, -1, -1,  1,  1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1,  1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0,  1, -1, -1, -1,  1,
        -1, -1, -1, -1,  1,  1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1,  1,  1, -1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1,  1, -1,  1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1, -1,  1, -1,  1,
        -1, -1, -1, -1,  1,  1], dtype=int8),
 array([ 1, -1,  1,  1, -1,  1,  1,  1,  0,  0,  0,  0, -1, -1,  1, -1,  1,
        -1, -1, -1, -1,  1, -1], dtype=int8),
 array([ 1, -1,  1,  1, 

In [24]:
max_num_objects = 2 ** m
max_num_objects
actual_num_objects / max_num_objects

max_num_partial_fvs = (2 + 1) ** m
# max_num_partial_fvs
'{:2,}'.format(max_num_partial_fvs)
'{:2E}'.format(max_num_partial_fvs)

8388608

1.0848045349121094e-05

'94,143,178,827'

'9.414318E+10'

# Operations 

## Make generator vectors

In [25]:
def make_generator_vectors(num_features):
    basis_vectors = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    basis_vectors_neg = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    for i,v in enumerate(basis_vectors):
        v[i] = 1
    for i,v in enumerate(basis_vectors_neg):
        v[i] = -1
    generators = basis_vectors + basis_vectors_neg
    return generators

In [26]:
generators = make_generator_vectors(m)
generators

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=int8),
 array([0,

In [27]:
# max_num_objects = 2 ** m
# max_num_objects

# max_num_partial_fvs = (2 + 1) ** m
# max_num_partial_fvs

## Boilerplate

In [28]:
def wf_pfv(v):
    '''
    Indicates whether v is a well-formed partially-specified feature vector.
    '''
    allowedValues = {-1,0,1}
    return all([x in allowedValues for x in v])

In [29]:
def wf_tfv(v):
    '''
    Indicates whether v is a well-formed totally-specified feature vector.
    '''
    allowedValues = {-1,1}
    return all([x in allowedValues for x in v])

In [30]:
def uniquify(ndarray_iterable):
    tuples = [tuple(a) for a in ndarray_iterable]
    s = set(tuples)
    arrays = [np.array(t) for t in s]
    return arrays

## Upper and lower closures of a partially-specified feature vector

In [31]:
upset_size_for_fsfvs = np.sum([binom(m, i) for i in np.arange(1,m)]); upset_size_for_fsfvs
"{:,}".format(upset_size_for_fsfvs)
"{:.2E}".format(upset_size_for_fsfvs)

8388606.0

'8,388,606.0'

'8.39E+06'

In [32]:
def put_along_axis_(arr, indices, values, axis=None, copy_arg=True):
    '''
    A functional version of np.put_along_axis that returns the 
    array it modifies.
    '''
    if copy_arg:
        my_arr = arr.copy()
    else:
        my_arr = arr
    np.put_along_axis(my_arr, indices, values, axis=axis)
    return my_arr

In [33]:
%timeit list(combinations(np.arange(20), 8))

10.4 ms ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
%timeit np.stack(combinations(np.arange(20), 8))

/home/AD/emeinhar/anaconda3/envs/jax-gpu2/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


379 ms ± 4.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit np.stack(tuple(combinations(np.arange(20), 8)))

382 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
from scipy.special import comb

def comb_index(n, k):
    count = comb(n, k, exact=True)
    index = np.fromiter(chain.from_iterable(combinations(range(n), k)), 
                        int, count=count*k)
    return index.reshape(-1, k)

In [37]:
%timeit comb_index(20, 8)

28.2 ms ± 592 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
np.array_equal( np.stack(combinations(np.arange(5), 3)),
                comb_index(5, 3) )

/home/AD/emeinhar/anaconda3/envs/jax-gpu2/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


True

In [39]:
import itertools
import timeit 

def nump(n, k, i=0):
    if k == 1:
        a = np.arange(i, i+n)
        return tuple([a[None, j:] for j in range(n)])
    template = nump(n-1, k-1, i+1)
    full = np.r_[np.repeat(np.arange(i, i+n-k+1),
                           [t.shape[1] for t in template])[None, :],
                 np.c_[template]]
    return tuple([full[:, j:] for j in np.r_[0, np.add.accumulate(
        [t.shape[1] for t in template[:-1]])]])

def nump2(n, k):
    a = np.ones((k, n-k+1), dtype=int)
    a[0] = np.arange(n-k+1)
    for j in range(1, k):
        reps = (n-k+j) - a[j-1]
        a = np.repeat(a, reps, axis=1)
        ind = np.add.accumulate(reps)
        a[j, ind[:-1]] = 1-reps[1:]
        a[j, 0] = j
        a[j] = np.add.accumulate(a[j])
    return a

def itto(L, N):
    return np.array([a for a in itertools.combinations(L,N)]).T

k = 6
n = 12
N = np.arange(n)

assert np.all(nump2(n,k) == itto(N,k))

print('numpy    ', timeit.timeit('f(a,b)', number=100, globals={'f':nump, 'a':n, 'b':k}))
print('numpy 2  ', timeit.timeit('f(a,b)', number=100, globals={'f':nump2, 'a':n, 'b':k}))
print('itertools', timeit.timeit('f(a,b)', number=100, globals={'f':itto, 'a':N, 'b':k}))

numpy     0.040146806044504046
numpy 2   0.013939313008449972
itertools 0.060256497003138065


In [40]:
%timeit nump(20, 8)[0]

3.59 ms ± 88.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%timeit nump2(20, 8)

10.8 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
def nump2_t(n, k):
    a = torch.ones((k, n-k+1), dtype=torch.int64)
    a[0] = torch.arange(n-k+1)
    for j in range(1,k):
        reps = (n-k+j) - a[j-1]
        a = torch.repeat_interleave(a, reps, dim=1)
        ind = torch.cumsum(reps, dim=0)
        a[j, ind[:-1]] = 1 - reps[1:]
        a[j, 0] = j
        a[j] = torch.cumsum(a[j], dim=0)
    return a

In [42]:
np.array_equal( comb_index(5, 3),
                nump(5,3)[0].T )

np.array_equal( comb_index(5, 3),
                nump2(5,3).T )

True

True

In [51]:
np_answer = nump2(5,3).T
np_answer.shape
torch_answer = nump2_t(5,3).t()
torch_answer.shape
torch.equal( torch.from_numpy(np_answer).to(device=torch_answer.device),
             torch_answer)

(10, 3)

torch.Size([10, 3])

True

In [52]:
%timeit nump2_t(20, 8).t()

2.18 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
put_along_axis_(np.zeros((int(binom(5,1)), 5), dtype=myint),
                nump2(5,1).T,
                1,
                axis=1,
                copy_arg=False)

put_along_axis_(np.zeros((int(binom(5,2)), 5), dtype=myint),
                nump2(5,2).T,
                1,
                axis=1,
                copy_arg=False)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]], dtype=int8)

array([[1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1]], dtype=int8)

In [55]:
# %%timeit

nump2(5, 1).T
nump2(5, 2).T
nump2(5, 3).T

# np.concatenate([nump2(5, 1).T, nump2(5, 2).T, nump2(5, 3).T])
# np.concatenate([nump2(5, 1).T, nump2(5, 2).T, nump2(5, 3).T]).shape

array([[0],
       [1],
       [2],
       [3],
       [4]])

array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4],
       [3, 4]])

array([[0, 1, 2],
       [0, 1, 3],
       [0, 1, 4],
       [0, 2, 3],
       [0, 2, 4],
       [0, 3, 4],
       [1, 2, 3],
       [1, 2, 4],
       [1, 3, 4],
       [2, 3, 4]])

In [95]:
def n_choose_at_most_k_indices_comb(n, k, asMask=True):
    my_f = nump
    my_k = k
    extra_step = False
    if my_f == nump:
        my_f = lambda n, i: nump(n,i)[0]
        my_k = k if k < n else k-1
        extra_step = False if k < n else True
        
    
    if not asMask:
        exact_results_indices = [my_f(n,i).T for i in np.arange(1, my_k+1)]
        return exact_results_indices
#         print(exact_results_indices)
    mask = np.concatenate([put_along_axis_(np.zeros((int(binom(n,i)), n), dtype=myint),
                                           my_f(n,i).T,
                                           1,
                                           axis=1,
                                           copy_arg=False)
                           for i in np.arange(1,my_k+1)])
    extra_step = False
    if extra_step:
#         print(mask.shape)
#         print(np.ones((n,)).shape)
#         mask = np.stack([mask, np.ones((n,), dtype=myint)], axis=1)
        mask = np.concatenate([mask, np.ones((1,n), dtype=myint)])
    return mask

In [57]:
n_choose_at_most_k_indices_comb(5,1)
# n_choose_at_most_k_indices_comb(5,1, False)
n_choose_at_most_k_indices_comb(5,2)
n_choose_at_most_k_indices_comb(5,2, False)
# n_choose_at_most_k_indices_comb(5,2).shape
binom(5,1) + binom(5,2)
n_choose_at_most_k_indices_comb(5,5)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]], dtype=int8)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1]], dtype=int8)

[array([[0],
        [1],
        [2],
        [3],
        [4]]), array([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 3],
        [2, 4],
        [3, 4]])]

15.0

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [1, 1, 1, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 0, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1],
       [1, 0, 0, 1, 1],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1],
       [0, 1, 1, 1, 1]], dtype=int8)

In [58]:
%%timeit

#nump
n_choose_at_most_k_indices_comb(20,8)

18.7 ms ± 646 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
%%timeit

#nump2
n_choose_at_most_k_indices_comb(20,8)

27.4 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
nump(5,3)[0].T

array([[0, 1, 2],
       [0, 1, 3],
       [0, 1, 4],
       [0, 2, 3],
       [0, 2, 4],
       [0, 3, 4],
       [1, 2, 3],
       [1, 2, 4],
       [1, 3, 4],
       [2, 3, 4]])

In [63]:
my_x = np.array([-1,0,1,0,1,-1,0,1,0]); my_x
my_x.shape
(my_x == 0).nonzero()[0]
my_x.nonzero()[0]

array([-1,  0,  1,  0,  1, -1,  0,  1,  0])

(9,)

array([1, 3, 6, 8])

array([0, 2, 4, 5, 7])

In [65]:
full_n = 8
dummy = np.zeros(8, dtype=myint)
# dummy2 = dummy.copy()
other_indices = np.array([1,3,6,8])
offsets = np.arange(len(other_indices))
my_indices = np.array([0,2,4,5,7])
# put_(dummy, my_indices, 1)
' '
n_choose_at_most_k_indices_comb(5,3)[-1]
my_indices[(n_choose_at_most_k_indices_comb(5,3)[-1]).astype('bool')]
n_choose_at_most_k_indices_comb(5,3).shape
np.insert(n_choose_at_most_k_indices_comb(5,3),
#           obj=other_indices,
          obj=other_indices - offsets,
#           values=np.zeros(other_indices.shape),
#           obj=1,
          values=0,
          axis=1
         )
' '
# (n_choose_at_most_k_indices_comb(5,3)[-1]).astype('bool')
# (n_choose_at_most_k_indices_comb(5,3)[-1]).astype('bool').shape
my_indices
# np.putmask(dummy2, (n_choose_at_most_k_indices_comb(5,3)[-1]).astype('bool'), np.ones((5,))); dummy2
# my_indices[None,:][(n_choose_at_most_k_indices_comb(5,3)).astype('bool')]
np.compress(n_choose_at_most_k_indices_comb(5,3)[-1],
            my_indices#,
           )

' '

array([0, 0, 1, 1, 1], dtype=int8)

array([4, 5, 7])

(25, 5)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 1, 0]], dtype=int8)

' '

array([0, 2, 4, 5, 7])

array([4, 5, 7])

In [66]:
my_x.shape
(my_x * np.logical_not(np.insert(n_choose_at_most_k_indices_comb(5,3),
                                obj=other_indices - offsets,
                                values=0,
                                axis=1)).astype(myint)).shape
my_x
my_x * np.logical_not(np.insert(n_choose_at_most_k_indices_comb(5,3),
                                obj=other_indices - offsets,
                                values=0,
                                axis=1)).astype(myint)

(9,)

(25, 9)

array([-1,  0,  1,  0,  1, -1,  0,  1,  0])

array([[ 0,  0,  1,  0,  1, -1,  0,  1,  0],
       [-1,  0,  0,  0,  1, -1,  0,  1,  0],
       [-1,  0,  1,  0,  0, -1,  0,  1,  0],
       [-1,  0,  1,  0,  1,  0,  0,  1,  0],
       [-1,  0,  1,  0,  1, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  1, -1,  0,  1,  0],
       [ 0,  0,  1,  0,  0, -1,  0,  1,  0],
       [ 0,  0,  1,  0,  1,  0,  0,  1,  0],
       [ 0,  0,  1,  0,  1, -1,  0,  0,  0],
       [-1,  0,  0,  0,  0, -1,  0,  1,  0],
       [-1,  0,  0,  0,  1,  0,  0,  1,  0],
       [-1,  0,  0,  0,  1, -1,  0,  0,  0],
       [-1,  0,  1,  0,  0,  0,  0,  1,  0],
       [-1,  0,  1,  0,  0, -1,  0,  0,  0],
       [-1,  0,  1,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  1,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  1,  0],
       [-1

In [67]:
nump(5,3)[0].T[0]
np.take(my_indices,
        nump(5,3)[0].T[0])
np.take(my_indices,
        nump(5,3)[0].T[0])

array([0, 1, 2])

array([0, 2, 4])

array([0, 2, 4])

In [68]:
n_choose_at_most_k_indices_comb(5,3, False)

[np.take(my_indices,#[None, :],
        each#,
#         axis=1
       ) for each in n_choose_at_most_k_indices_comb(5,3, False)]

[array([[0],
        [1],
        [2],
        [3],
        [4]]), array([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 3],
        [2, 4],
        [3, 4]]), array([[0, 1, 2],
        [0, 1, 3],
        [0, 1, 4],
        [0, 2, 3],
        [0, 2, 4],
        [0, 3, 4],
        [1, 2, 3],
        [1, 2, 4],
        [1, 3, 4],
        [2, 3, 4]])]

[array([[0],
        [2],
        [4],
        [5],
        [7]]), array([[0, 2],
        [0, 4],
        [0, 5],
        [0, 7],
        [2, 4],
        [2, 5],
        [2, 7],
        [4, 5],
        [4, 7],
        [5, 7]]), array([[0, 2, 4],
        [0, 2, 5],
        [0, 2, 7],
        [0, 4, 5],
        [0, 4, 7],
        [0, 5, 7],
        [2, 4, 5],
        [2, 4, 7],
        [2, 5, 7],
        [4, 5, 7]])]

In [69]:
def combine(u, v):
#     M = np.stack([u,v])
    return np.logical_or(u, v).astype(myint)

combine_ = np.vectorize(combine, 
                        signature='(n),(m,n)->(m,n)')

n = 5
b1 = np.eye(n, dtype=myint); b1
np.concatenate(combine_(b1, b1)).shape
np.unique(np.concatenate(combine_(b1, b1)), axis=0)

def combine_acc(b, acc, k):
    if k == 1:
#         return b
        return acc
    elif k == 2:
        return np.unique(np.concatenate(combine_(b, 
                                                 acc)), 
                         axis=0)
    #block left here for illustrative purposes...
#     elif k == 3:
#         b2 = np.unique(np.concatenate(combine_(b, 
#                                                acc)), 
#                        axis=0)
#         return np.unique(np.concatenate(combine_(b,
#                                                  b3)),
#                          axis=0)
#         return np.unique(np.concatenate(combine_(b, 
#                                                  combine_acc(b, acc, k-1))), 
#                          axis=0)
    else:
        return np.unique(np.concatenate(combine_(b,
                                                 combine_acc(b, acc, k-1))),
                         axis=0)
#     return np.unique(np.concatenate(combine_acc()))

def n_choose_at_most_k_indices(n, k):
    '''
    Returns 𝚺_i=1^i=k n choose i vectors representing 
    all ways of selecting i elements from a vector of length n,
    for all i from 1 to k (inclusive).
    '''
    b1 = np.eye(n, dtype=myint)
#     B = np.empty()
  
#     return combine_acc(b1, b1, k)
    assert k > 0, 'k must be greater than 0'
    
    acc = b1.copy()
    while k > 1:
        acc = np.unique(np.concatenate(combine_(b1, acc)),
                        axis=0)
        k-=1
    del k
    return acc
        
    
#     indices = np.arange(n)
#     CP_of_indices = np.array(np.meshgrid(*np.tile(indices, (k,1)))).T.reshape(-1,k)
#     return CP_of_indices
' '
n_choose_at_most_k_indices(5, 2)
n_choose_at_most_k_indices(5, 2).shape
np.sum([binom(5,i) for i in np.arange(1,3)])
# b1.shape
# combine_(b1, n_choose_at_most_k_indices(5, 2))
# np.unique(np.concatenate(combine_(b1, n_choose_at_most_k_indices(5, 2))), axis=0)
n_choose_at_most_k_indices(5, 3)
n_choose_at_most_k_indices(5, 3).shape
np.sum([binom(5,i) for i in np.arange(1,4)])

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]], dtype=int8)

(25, 5)

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int8)

' '

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int8)

(15, 5)

15.0

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [1, 1, 0, 1, 0],
       [1, 1, 1, 0, 0]], dtype=int8)

(25, 5)

25.0

In [ ]:
%%timeit

n_choose_at_most_k_indices(20, 8)

In [ ]:
# np.frompyfunc(combine_, 2, 1).reduce(b1, axis=0, keepdims=False, initial=b1)

In [70]:
# def combine(u, v):
#     return (u | v).astype(myint)

def combine(u, v):
#     M = np.stack([u,v])
    return np.logical_or(u, v).astype(myint)

combine_ = np.vectorize(combine, 
#                         otypes='np.int8', 
                        signature='(m),(m,n)->(m,n)')

In [71]:
b1[0]
b1[1]
' '
combine(b1[0], b1[0])
combine(b1[0], b1[1])
combine(b1[0], b1)
combine_(b1[0], b1)
combine_(b1, b1)
combine_(b1, b1).shape
np.concatenate(combine_(b1, b1))
np.concatenate(combine_(b1, b1)).shape
np.unique(np.concatenate(combine_(b1, b1)),
          axis=0)
# combine_(b1[0], b1)

array([1, 0, 0, 0, 0], dtype=int8)

array([0, 1, 0, 0, 0], dtype=int8)

' '

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 1, 0, 0, 0], dtype=int8)

...

array([[[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1]],

       [[1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1]],

       [[1, 0, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 1, 0],
        [0, 0, 1, 0, 1]],

       [[1, 0, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1]],

       [[1, 0, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 0, 1]]], dtype=int8)

(5, 5, 5)

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1],
       [1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1]], dtype=int8)

(25, 5)

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int8)

In [90]:
b1[0].shape
b1.shape
np.logical_or(b1[0][None, :], b1).astype(myint)
np.logical_or(b1[0][None, :], b1, out=np.zeros((4,4))).astype(myint)
np.logical_or(b1[None, :, :], b1[None, :, :]).astype(myint)

(4,)

(4, 4)

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 0, 1]], dtype=int8)

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 0, 1]], dtype=int8)

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]], dtype=int8)

In [73]:
n = 4
z = np.zeros(n, dtype=myint); z # <- all ways of choosing 0 objects from a set of n
b1 = np.eye(n, dtype=myint); b1 # <- all ways of choosing 1 distinct object from a set of n
# np.outer(b,b).astype(myint)

'---'
np.roll(b1, 1, 0)
b1 + np.roll(b1, 1, 0)
# np.roll(np.roll(b1, 1, 0), 1, 0) 
np.roll(b1, 2, 0)
b1 + np.roll(b1, 2, 0)
np.roll(b1, 3, 0)
b1 + np.roll(b1, 3, 0)
# np.roll(b1, 4, 0)

# b2 = b1 + b1.transpose(); b2
# b2 = np.einsum('ij,ij->ij', b1, b1); b2
'---'

nb1 = np.logical_not(b1); nb1.astype(myint) #<- all ways of choosing n-1 distinct objects from a set of n
# np.matmul(nb, nb).astype(myint)
np.unique(b1 + nb1, axis=0) #<- all ways of choosing ((n-1) + 1) = n distinct objects from a set of n
' '
# np.dot(b,nb)
# np.matmul(b,nb)
# np.outer(b,nb)#.shape
# np.kron(b,nb)

array([0, 0, 0, 0], dtype=int8)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]], dtype=int8)

'---'

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]], dtype=int8)

array([[1, 0, 0, 1],
       [1, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 1]], dtype=int8)

...

array([[1, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 1],
       [1, 0, 0, 1]], dtype=int8)

'---'

array([[0, 1, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 0]], dtype=int8)

array([[1, 1, 1, 1]], dtype=int8)

' '

In [75]:
# b[0] + b[1]
# b[0] + b[2]
# b[1] + b[2]

In [76]:
binom(20,8)

125970.0

In [77]:
np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7])).T.reshape(-1,3)

array([[1, 4, 6],
       [1, 5, 6],
       [2, 4, 6],
       [2, 5, 6],
       [3, 4, 6],
       [3, 5, 6],
       [1, 4, 7],
       [1, 5, 7],
       [2, 4, 7],
       [2, 5, 7],
       [3, 4, 7],
       [3, 5, 7]])

In [78]:
np.array(np.meshgrid([0, 1, 2], [0, 1, 2])).T.reshape(-1,2)
np.array(np.meshgrid([0, 1, 2], [0, 1, 2])).T.reshape(-1,2).shape
np.array(np.meshgrid(*[[0, 1, 2], [0, 1, 2]])).T.reshape(-1,2)
np.array(np.meshgrid(*np.array([[0, 1, 2], [0, 1, 2]]))).T.reshape(-1,2)
np.array(np.meshgrid([0, 1, 2], [0, 1, 2], [0, 1, 2])).T.reshape(-1,3)
np.array(np.meshgrid([0, 1, 2], [0, 1, 2], [0, 1, 2])).T.reshape(-1,3).shape

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

(9, 2)

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

array([[0, 0, 0],
       [0, 1, 0],
       [0, 2, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 2, 0],
       [2, 0, 0],
       [2, 1, 0],
       [2, 2, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 2, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 2, 1],
       [2, 0, 1],
       [2, 1, 1],
       [2, 2, 1],
       [0, 0, 2],
       [0, 1, 2],
       [0, 2, 2],
       [1, 0, 2],
       [1, 1, 2],
       [1, 2, 2],
       [2, 0, 2],
       [2, 1, 2],
       [2, 2, 2]])

(27, 3)

In [96]:
def put_(a, ind, v, mode='raise', copy_arg=True):
    '''
    A functional version of np.put that returns the array it operates on. 
    See the documentation for that function for more details.
    '''
    if copy_arg:
        my_a = a.copy()
    else:
        my_a = a
    np.put(a=my_a, ind=ind, v=v, mode=mode)
    return my_a


def upper_closure(x, astype='ndarray'):
    '''
    The upper closure ↑x of a pfv x is the set of strictly less specified vectors.
    
    
    WARNING: There are O(𝚺_i=1^i=m m choose i) elements in this set.
    '''
    specified_indices = x.nonzero()[0]
    m_x = len(specified_indices)
    
    unspecified_indices = (x == 0).nonzero()[0]
    offsets = np.arange(len(unspecified_indices))
    
    #There is one element in ↑x for each possible combination of specified indices.
    
    if astype == 'ndarray':
    #     print(specified_indices, m_x)
        combinations_of_indices_to_unspecify = n_choose_at_most_k_indices_comb(m_x, m_x, True)
        mask = np.insert(combinations_of_indices_to_unspecify,
                         obj = unspecified_indices - offsets,
                         values = 0,
                         axis = 1)
        eraser_mask = np.logical_not(mask).astype(myint)
        return x * eraser_mask.astype(myint)
#         combinations_of_indices_to_unspecify = [np.take(specified_indices,
#                                                         each)
#                                                 for each in n_choose_at_most_k_indices_comb(m_x, m_x, False)]

#         print(np.sum(binom(m_x, i) for i in np.arange(1, m_x)))
#         print(combinations_of_indices_to_unspecify.shape)
#         print(x.shape)
#         up_x = put_along_axis_(x[None, :], 
#                                combinations_of_indices_to_unspecify,
#                                0,
#                                axis=1,
#                                copy_arg=False)
    elif astype == 'generator':
    #     #5-10x slower
        combinations_of_indices_to_unspecify = cat(combinations(specified_indices, i)
                                                   for i in range(1,m_x))
        up_x = (put_(x, tuple(ind), 0) for ind in combinations_of_indices_to_unspecify)
        return up_x
    else:
        raise Exception(f"astype must be either 'generator' or 'ndarray'")

# def upper_closure_t(x):
#     '''
#     The upper closure ↑x of a pfv x is the set of strictly less specified vectors.
#     This function returns that as a generator.
    
#     WARNING: There are O(𝚺_i=1^i=m m choose i) elements in this set.
#     '''
#     specified_indices = x.nonzero()[0]
#     m_x = len(specified_indices)
    
#     unspecified_indices = (x == 0).nonzero()[0]
#     offsets = torch.arange(len(unspecified_indices))
    
#     #There is one element in ↑x for each possible combination of specified indices.
    
# #     print(specified_indices, m_x)
#     combinations_of_indices_to_unspecify = n_choose_at_most_k_indices_comb(m_x, m_x, True)
#     mask = np.insert(combinations_of_indices_to_unspecify,
#                      obj = unspecified_indices - offsets,
#                      values = 0,
#                      axis = 1)
#     eraser_mask = np.logical_not(mask).astype(myint)
#     return x * eraser_mask.astype(myint)


def lower_closure(x):
    '''
    The lower closure ↓x of a pfv x is the set of strictly more specified vectors.
    This function returns that as a generator.
    
    WARNING: There are O(𝚺_i=1^i=m choose(m,i) * 2^i) elements in this set.
    '''
    unspecified_indices = (x == 0).nonzero()[0]
    m_x = len(unspecified_indices)
    #There are 2^i elements in ↓x for each possible combination of i unspecified indices.
    combinations_of_indices_to_specify = cat(combinations(unspecified_indices, i)
                                             for i in range(1,m_x))
#     specifications = cat(map(np.array, permutations([-1,1], len(combo)))
#                          for combo in combinations_of_indices_to_specify)
    down_x = (put_(x, tuple(ind), spec) 
              for ind in combinations_of_indices_to_specify
              for spec in map(np.array, 
                              product([-1,1], repeat=len(ind))))
    return down_x


def gen_uc(x):
    '''
    Generates a random element u of ↑x.
    Generative procedure:
      1. A number n of indices to unspecify is chosen uniformly from among specified ones.
      2. n indices are sampled without replacement from among the specified ones.
    '''
    specified_indices = x.nonzero()[0]
    m_x = len(specified_indices)
    num_indices_to_unspecify = choice(np.arange(1,m_x))
#     assert num_indices_to_unspecify > 0
    indices_to_unspecify = np.random.choice(specified_indices, 
                                            size=num_indices_to_unspecify, 
                                            replace=False)
    u = put_(x, indices_to_unspecify, 0)
    return u


def gen_lc(x):
    '''
    Generates a random element l of ↓x.
    Generative procedure:
      1. A number n of indices to specify is chosen uniformly from among unspecified ones.
      2. n indices are sampled without replacement from among the unspecified ones.
    '''
    unspecified_indices = (x == 0).nonzero()[0]
    m_x = len(unspecified_indices)
    num_indices_to_specify = choice(np.arange(1,m_x))
#     assert num_indices_to_specify > 0
    indices_to_specify = np.random.choice(unspecified_indices, 
                                         size=num_indices_to_specify, 
                                         replace=False)
    possible_specifications = lmap(np.array, product([-1,1], repeat=len(indices_to_specify)))
    if len(possible_specifications) == 0:
        print(x, m_x, unspecified_indices, num_indices_to_specify, indices_to_specify)
    spec = choice(possible_specifications)
    l = put_(x, indices_to_specify, spec)
    return l


def gen_agreeing(x):
    '''
    Generates a random psfv vector r that agrees with x.
    '''
    specified_indices = x.nonzero()[0]
    unspecified_indices = (x == 0).nonzero()[0]
    has_uc = len(specified_indices) > 0
    has_lc = len(unspecified_indices) > 0
    if has_uc and has_lc:
        sample_function = choice([gen_uc, gen_lc])
        return sample_function(x)
    elif has_uc:
        return gen_uc(x)
    elif has_lc:
        return gen_lc(x)
    else:
        raise Exception(f'x has neither an upper nor a lower closure:\n\tx = {x}')

In [84]:
r = choice(objects); r

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

In [99]:
%%timeit

list(upper_closure(choice(objects), 'generator'))

785 ms ± 232 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [97]:
r
ucr = upper_closure(r, 'generator')
ucr_l = list(ucr)
ucr_old = np.stack(ucr_l)
ucr_old.shape
# len(ucr_l)
ucr_l[0]
ucr_l[0].shape
choice(ucr_l)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

(16382, 23)

array([ 0,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

(23,)

array([-1,  1, -1,  0, -1, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,
       -1, -1,  0,  0,  0,  0], dtype=int8)

In [100]:
%%timeit

#
list(upper_closure(choice(objects), 'ndarray'))

132 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [101]:
r
ucr_new = upper_closure(r, 'ndarray')
ucr_new.shape
ucr_new[0]
ucr_new[0].shape
choice(ucr_new)

np.array_equal(ucr_old, ucr_new)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

(16382, 23)

array([ 0,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

(23,)

array([ 0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1,  0,  0,  0,  0], dtype=int8)

True

In [105]:
set(map(lambda arr: tuple([x for x in arr.squeeze()]),
        np.split(ucr_old, ucr_old.shape[0]))) - set(map(lambda arr: tuple([x for x in arr.squeeze()]), 
                                                        np.split(ucr_new, ucr_new.shape[0])))
set(map(lambda arr: tuple([x for x in arr.squeeze()]), 
        np.split(ucr_new, ucr_new.shape[0]))) - set(map(lambda arr: tuple([x for x in arr.squeeze()]),
                                                        np.split(ucr_old, ucr_old.shape[0])))

set()

set()

In [146]:
upper_closures = [upper_closure(o)
                  for o in tqdm(objects)]
lmap(lambda uc: uc.shape, upper_closures)
num_elements_UC = sum(lmap(lambda uc: uc.shape[0], upper_closures))
print('Not accounting for overlap...')
"{:.2E}".format(num_elements_UC)
" = {:.2E}% of all logically possible pfvs".format((num_elements_UC / 3 ** m) * 100)
"{:,}".format(num_elements_UC)
" = {:,}% of all logically possible pfvs".format((num_elements_UC / 3 ** m) * 100)

for uc in upper_closures:
    uc.flags.writeable = False
    
upper_closures_dict = {hash(o.tostring()):upper_closure(o)
                       for o in tqdm(objects)}



  0%|          | 0/91 [00:00<?, ?it/s]

  4%|▍         | 4/91 [00:00<00:03, 27.18it/s]

  7%|▋         | 6/91 [00:00<00:03, 23.33it/s]

  9%|▉         | 8/91 [00:00<00:04, 17.33it/s]

 15%|█▌        | 14/91 [00:00<00:03, 20.19it/s]

 18%|█▊        | 16/91 [00:00<00:04, 16.00it/s]

 20%|█▉        | 18/91 [00:00<00:04, 15.48it/s]

 23%|██▎       | 21/91 [00:01<00:03, 17.63it/s]

 25%|██▌       | 23/91 [00:01<00:04, 16.55it/s]

 30%|██▉       | 27/91 [00:01<00:03, 19.56it/s]

 33%|███▎      | 30/91 [00:01<00:02, 20.88it/s]

 38%|███▊      | 35/91 [00:01<00:02, 23.14it/s]

 42%|████▏     | 38/91 [00:01<00:02, 23.48it/s]

 47%|████▋     | 43/91 [00:01<00:01, 24.52it/s]

 51%|█████     | 46/91 [00:02<00:02, 22.32it/s]

 54%|█████▍    | 49/91 [00:02<00:01, 23.54it/s]

 57%|█████▋    | 52/91 [00:02<00:01, 24.92it/s]

 60%|██████    | 55/91 [00:02<00:01, 21.56it/s]

 64%|██████▎   | 58/91 [00:02<00:01, 21.29it/s]

 67%|██████▋   | 61/91 [00:02<00:01, 22.10it/s]

 70%|███████   | 64/91 [00:02<

[(16382, 23),
 (131070, 23),
 (131070, 23),
 (524286, 23),
 (524286, 23),
 (131070, 23),
 (524286, 23),
 (524286, 23),
 (32766, 23),
 (32766, 23),
 (131070, 23),
 (131070, 23),
 (131070, 23),
 (524286, 23),
 (524286, 23),
 (524286, 23),
 (524286, 23),
 (262142, 23),
 (32766, 23),
 (65534, 23),
 (524286, 23),
 (524286, 23),
 (262142, 23),
 (262142, 23),
 (16382, 23),
 (131070, 23),
 (262142, 23),
 (524286, 23),
 (16382, 23),
 (131070, 23),
 (131070, 23),
 (131070, 23),
 (131070, 23),
 (8190, 23),
 (524286, 23),
 (131070, 23),
 (32766, 23),
 (524286, 23),
 (131070, 23),
 (16382, 23),
 (65534, 23),
 (262142, 23),
 (524286, 23),
 (262142, 23),
 (131070, 23),
 (524286, 23),
 (524286, 23),
 (32766, 23),
 (32766, 23),
 (32766, 23),
 (262142, 23),
 (262142, 23),
 (262142, 23),
 (524286, 23),
 (262142, 23),
 (524286, 23),
 (16382, 23),
 (262142, 23),
 (524286, 23),
 (16382, 23),
 (131070, 23),
 (524286, 23),
 (131070, 23),
 (262142, 23),
 (262142, 23),
 (524286, 23),
 (262142, 23),
 (524286, 23

Not accounting for overlap...


'2.28E+07'

' = 2.42E-02% of all logically possible pfvs'

'22,765,386'

' = 0.024181662743547546% of all logically possible pfvs'



  0%|          | 0/91 [00:00<?, ?it/s]

  4%|▍         | 4/91 [00:00<00:03, 28.04it/s]

  7%|▋         | 6/91 [00:00<00:03, 23.75it/s]

  9%|▉         | 8/91 [00:00<00:04, 17.08it/s]

 15%|█▌        | 14/91 [00:00<00:03, 20.05it/s]

 18%|█▊        | 16/91 [00:00<00:04, 15.47it/s]

 20%|█▉        | 18/91 [00:00<00:04, 15.09it/s]

 23%|██▎       | 21/91 [00:01<00:04, 17.10it/s]

 25%|██▌       | 23/91 [00:01<00:04, 15.65it/s]

 30%|██▉       | 27/91 [00:01<00:03, 18.52it/s]

 33%|███▎      | 30/91 [00:01<00:03, 19.74it/s]

 38%|███▊      | 35/91 [00:01<00:02, 21.87it/s]

 42%|████▏     | 38/91 [00:01<00:02, 22.24it/s]

 47%|████▋     | 43/91 [00:01<00:02, 23.47it/s]

 51%|█████     | 46/91 [00:02<00:02, 21.46it/s]

 54%|█████▍    | 49/91 [00:02<00:01, 22.58it/s]

 57%|█████▋    | 52/91 [00:02<00:01, 23.97it/s]

 60%|██████    | 55/91 [00:02<00:01, 20.15it/s]

 64%|██████▎   | 58/91 [00:02<00:01, 19.64it/s]

 67%|██████▋   | 61/91 [00:02<00:01, 20.36it/s]

 70%|███████   | 64/91 [00:03<

In [139]:
upper_closures_dict[hash(objects[3].tostring())]

array([[ 0, -1,  1, ..., -1,  1,  1],
       [ 1,  0,  1, ..., -1,  1,  1],
       [ 1, -1,  0, ..., -1,  1,  1],
       ...,
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 0, -1,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0]], dtype=int8)

In [150]:
all_uc = np.concatenate(upper_closures)
all_uc.shape
all_uc.dtype
all_uc_t = torch.from_numpy(all_uc)
if torch.cuda.is_available():
    all_uc_tc = all_uc_t.cuda()

(22765386, 23)

dtype('int8')

In [149]:
#130s wittgenstein/brh
pfvs_with_nonempty_extension = np.unique(all_uc,
                                         axis=0)
pfvs_with_nonempty_extension.shape

(9115021, 23)

In [154]:
#120s wittgenstein/brh
pfvs_with_nonempty_extension_t = torch.unique(all_uc_t, 
                                              dim=0)

In [155]:
if torch.cuda.is_available():
    #8s wittgenstein/brh
    pfvs_with_nonempty_extension_tc = torch.unique(all_uc_tc, 
                                                   dim=0)

In [107]:
r
lcr = lower_closure(r)
lcr_l = list(lcr)
len(lcr_l)
lcr_l[0]
choice(lcr_l)
choice(lcr_l)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

19170

array([-1,  1, -1,  1, -1, -1,  1, -1, -1, -1,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1, -1,  1,  0,  0,  1,  1,  0, -1,
       -1, -1, -1, -1,  1, -1], dtype=int8)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  1,  0,  1,  0, -1,  1,  1, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

In [109]:
%%timeit

#
list(lower_closure(choice(objects)))

13.7 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [108]:
r
gen_uc(r)
gen_lc(r)
gen_agreeing(r)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

array([-1,  1, -1,  1, -1,  0,  1, -1, -1,  0,  0,  0,  0,  0,  0,  0, -1,
       -1,  0, -1, -1,  0,  0], dtype=int8)

array([-1,  1, -1,  1, -1, -1,  1, -1, -1, -1,  0, -1, -1,  1,  1,  1, -1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

array([ 0,  1,  0,  1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0, -1,  0,  0], dtype=int8)

## Agreement

In [156]:
def ag(x,y):
    '''
    Formula:
    (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y)), where T = 1 and F = 0
    
    Pattern:
    x = x ⟶ 1
    0 = _ ⟶ 1
    _ = 0 ⟶ 1
    _ = _ ⟶ 0
    '''
    if x == y:
        return True
    elif x == 0:
        return True
    elif y == 0:
        return True
    else:
        return False

In [157]:
def ag_(x,y):
#     return (not x*y) == -1 # <- BAD
    return not (x*y == -1)

In [158]:
ag(0,0) == ag_(0,0)
ag(0,1) == ag_(0,1)
ag(0,-1) == ag_(0,-1)
ag(-1,0) == ag_(-1,0)
ag(-1,1) == ag_(-1,1)
ag(-1,-1) == ag_(-1,-1)
ag(1,0) == ag_(1,0)
ag(1,1) == ag_(1,1)
ag(1,-1) == ag_(1,-1)

True

True

True

True

True

True

True

True

True

In [159]:
def agree(u,v):
    '''
    Given two vectors u and v, returns a binary vector indicating,
    elementwise, whether u and v 'agree'.
    
    agree(u[i], v[i]) iff (u[i] == 0 or v[i] == 0) or (u[i] == v[i])
    '''
#     return np.array([True if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else False 
#                      for i in range(len(u))])
    return np.array([1 if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else 0 
                     for i in range(len(u))], dtype=myint)

In [160]:
def agree_(u,v):
    '''
    Given two vectors u and v, return 1 iff u and v agree at all indices
    and 0 otherwise.
    '''
    ag = agree(u,v)
    return int(ag.all())

In [161]:
def agree_v(u,v):
    return (~(u*v == -1)).all()
#     return (not (u*v == -1)).all() #raises ambiguity error
#     return not (u*v == -1).all()

In [162]:
def agree_mat(A,B):
    '''
    Given two matrices A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = np.equal(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
    result = np.prod(ag, axis=-1, dtype=myint)
    return result

In [163]:
def agree_m(A, B, axis=0):
    return (~np.equal(A*B, -1)).prod(axis=axis)

In [164]:
# twice as slow as pytorch-cpu. not worth it.
# def agree_mat_ne(A,B):
#     '''
#     Given two matrices A,B :: (n,m)
#     return C::(n,1) where 
#     C[i] = 1 iff A[i] + B[i] agree at all indices,
#     returning 0 otherwise.
#     '''

#     # (x == 0 | y == 0) | ((x != 0 & y != 0) & (x == y))
#     A_unspecified = ne.evaluate('A == 0')
#     B_unspecified = ne.evaluate('B == 0')
#     A_v_B_unspecified = ne.evaluate('A_unspecified | B_unspecified')
    
#     A_specified = ne.evaluate('A != 0')
#     B_specified = ne.evaluate('B != 0')
#     both_A_B_specified = ne.evaluate('A_specified & B_specified')
#     A_equal_B = np.equal(A,B)
#     A_B_both_specified_also_equal = ne.evaluate('both_A_B_specified & A_equal_B')
    
#     ag = ne.evaluate('A_v_B_unspecified | A_B_both_specified_also_equal')
# #     return ag
# #     last_axis = ag.ndim-1
# #     a = last_axis
# #     result = ne.evaluate('prod(ag, axis=1)').astype(myint) #this manually setting it to a specific value still causes a bizarre error, just a different one
# #     result = ne.evaluate('prod(ag, axis=a)').astype(myint) #axis can't be a variable or else it causes a bizarre error
# #     result = ne.evaluate('prod(ag, axis=last_axis)').astype(myint) #axis can't be a variable or else it causes a bizarre error
#     result = np.prod(ag, axis=-1, dtype=myint)
#     return result

In [165]:
def agree_mat_t(A,B):
    '''
    Given two matrices (torch tensors) A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = torch.eq(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
#     result = np.prod(ag, axis=-1, dtype=myint)
    result = torch.zeros([A.shape[0]], dtype=my_dtype, device=A.device)
    result = torch.prod(ag, dim=1,dtype=my_dtype, out=result)
#     result = ag.type(torch.cuda.ByteTensor).all()
    if result.device.type == 'cuda':
        torch.cuda.empty_cache()
    return result#.type(my_torch_type)

In [166]:
def agree_mt(A, B, dim=0):
    return (~torch.eq(A*B, -1 * torch.ones(A.shape, dtype=A.dtype, device=A.device))).prod(dim=dim)

In [167]:
#note that this scales *poorly* with the number of features m

# Given that each feature's value is sampled iid and uniformly,
# the probability that two randomly generated features *disagree*
# is 2/9 = p('+-' ∨ '-+'), so the probability of *agreement* is 7/9.
# Therefore the probability of two random feature vectors with m features
# agreeing on all features is (7/9)^m

(7/9)**m

def make_agreeing_vector_pair(pred=None):
    u = make_random_pfv()
    v = make_random_pfv()
    if pred is None:
        while not agree_(u,v):
            u = make_random_pfv()
            v = make_random_pfv()
        return u,v
    while not agree_(u,v) and not pred(u,v):
        u = make_random_pfv()
        v = make_random_pfv()
    return u,v

0.0030879993711559394

In [168]:
num_test_pairs = int(1e5)
# random_vector_pairs = [(make_random_pfv(), make_random_pfv()) for each in range(num_test_pairs)]
random_vector_pairs = [(choice(objects), choice(objects)) for each in range(num_test_pairs)]
len(random_vector_pairs)

100000

In [169]:
num_test_pairs = int(1e5)
# agreeing_vector_pairs = [make_agreeing_vector_pair() for each in range(num_test_pairs)]
agreeing_vector_pairs = []
for each in range(num_test_pairs):
    obj = choice(objects)
    ag_obj = gen_agreeing(obj)
    agreeing_vector_pairs.append((obj, ag_obj))
len(agreeing_vector_pairs)

100000

In [170]:
# first = lambda seq: seq[0]
# second = lambda seq: seq[1]

stack_a, stack_b = lmap(first, random_vector_pairs), lmap(second, random_vector_pairs)
random_pair_stack_a, random_pair_stack_b = np.array(stack_a), np.array(stack_b)
random_pair_stack_a.dtype
random_pair_stack_b.dtype

random_pair_stack_a_t, random_pair_stack_b_t = torch.from_numpy(random_pair_stack_a.astype(np.int32)).type(torch.int8), torch.from_numpy(random_pair_stack_b.astype(np.int32)).type(torch.int8)

if torch.cuda.is_available():
    random_pair_stack_a_tc, random_pair_stack_b_tc = random_pair_stack_a_t.cuda(), random_pair_stack_b_t.cuda()

dtype('int8')

dtype('int8')

In [171]:
stack_a, stack_b = lmap(first, agreeing_vector_pairs), lmap(second, agreeing_vector_pairs)
agreeing_pair_stack_a, agreeing_pair_stack_b = np.array(stack_a), np.array(stack_b)
agreeing_pair_stack_a.dtype
agreeing_pair_stack_b.dtype

dtype('int8')

dtype('int8')

In [172]:
%%timeit

list(starmap(agree_, random_vector_pairs));

8.07 s ± 54.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [173]:
%%timeit

list(starmap(agree_v, random_vector_pairs));

400 ms ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [174]:
assert list(starmap(agree_, random_vector_pairs)) == list(starmap(agree_v, random_vector_pairs))

In [175]:
%%timeit

list(starmap(agree_m, random_vector_pairs));

538 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [176]:
assert list(starmap(agree_v, random_vector_pairs)) == list(starmap(agree_m, random_vector_pairs))

In [177]:
%%timeit

agree_mat(random_pair_stack_a, random_pair_stack_b)

5.91 ms ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [178]:
%%timeit

agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)

4.97 ms ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [179]:
%%timeit

agree_mt(random_pair_stack_a_t, random_pair_stack_b_t, dim=1)

8.05 ms ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [180]:
if torch.cuda.is_available():
    %timeit agree_mt(random_pair_stack_a_t.cuda(), random_pair_stack_b_t.cuda(), dim=1)

766 µs ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [181]:
if torch.cuda.is_available():
    %timeit agree_mt(random_pair_stack_a_tc, random_pair_stack_b_tc, dim=1)

271 µs ± 557 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [182]:
assert (agree_mat(random_pair_stack_a, random_pair_stack_b) == agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)).all()
assert (agree_mt(random_pair_stack_a_t, random_pair_stack_b_t, dim=1).numpy() == agree_m(random_pair_stack_a, random_pair_stack_b, axis=1)).all()

In [183]:
# %%timeit

# agree_mat_ne(random_pair_stack_a, random_pair_stack_b)

In [184]:
%%timeit

agree_mat_t(random_pair_stack_a_t, random_pair_stack_b_t)

29.1 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [185]:
if torch.cuda.is_available():
    %timeit agree_mat_t(random_pair_stack_a_tc, random_pair_stack_b_tc)

426 µs ± 630 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [186]:
np.array_equal(agree_mat(random_pair_stack_a, random_pair_stack_b), 
               list(starmap(agree_, random_vector_pairs)))

True

In [187]:
n = num_test_pairs
for i in range(n):
    u = random_pair_stack_a[i]
    v = random_pair_stack_b[i]
    assert agree_(u,v) == agree_mat(u,v), '{0}, {1} -> {2} vs. {3}'.format(u,v, agree_(u,v), agree_mat(u,v, True))

In [188]:
if torch.cuda.is_available():
    agreement = agree_mat_t
else:
    agreement = agree_mat

## Comparison of pfvs (by specification = without reference to actual extension)

In [189]:
# from https://docs.python.org/3.6/howto/sorting.html
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

In [190]:
def compare_specification_magnitude(u, v):
    '''
    Given two pfvs u, v, returns 
        1 iff u > v
        0 iff u == v
        -1 iff u < v
    where
        u > v
    iff u has *more* specified features
    than v.
    '''
    if type(u) == np.ndarray and type(v) == np.ndarray:
        backend = np
    elif type(u) == torch.Tensor and type(v) == torch.Tensor:
        backend = torch
    else:
        raise Exception('u,v must both either be of type np.ndarray or torch.Tensor')
    
    u_key, v_key = backend.sum(backend.abs(u)), backend.sum(backend.abs(v))
    if u_key == v_key:
        return 0
    else:
        if u_key > v_key:
            return 1
        else:
            return -1
    
compare_specification_magnitude(np.array([0,1,-1]),
                                np.array([0,1,-1]))

compare_specification_magnitude(np.array([0,1,0]),
                                np.array([0,1,-1]))

0

-1

In [191]:
agree_m(np.array([0,1,0]),
        np.array([0,1,1]))

agree_m(np.array([-1,1,0]),
        np.array([0,1,1]))

agree_m(np.array([0,1,-1]),
        np.array([0,1,1]))

1

1

0

In [193]:
def incomparable_features(a,b):
    return (a == -1 & b == 1) | (a == 1 & b == -1)

# incomparable_vec = np.vectorize(incomparable_features)

def incomparable(u,v):
    return (np.equal(u, -1) & np.equal(v, 1)) | (np.equal(u, 1) & np.equal(v, -1))
#     return incomparable_vec(u,v)

def comp_spec_feature(a, b):
    '''
    At the level of a single feature value f
         f ⊆ f, ∀f ∈ {-1,0,+1}
        +1 ⊂ 0
        -1 ⊂ 0
        (-1 and +1 are incomparable)
    
    This function returns 
         0    if a ⊆ b and b ⊆ a
        +1    if a ⊂ b
        -1    if b ⊂ a
        NaN   if a and b are incomparable
    '''
    if a == b:
        return 0
    elif a == 0:
        return 1
    elif b == 0:
        return -1
    else:
        return np.NaN
    
comp_spec_feature_vec = np.vectorize(comp_spec_feature)
    
def compare_spec(u,v):
    '''
    At the level of a single feature value f
         f ⊆ f, ∀f ∈ {-1,0,+1}
        +1 ⊂ 0
        -1 ⊂ 0
        (-1 and +1 are incomparable)
    
    This function returns 
         0    if u ⊆ v and v ⊆ u
        +1    if u ⊂ v
        -1    if v ⊂ u
        NaN   if u and v are incomparable
    *elementwise*
    
    '''
    incomparability = incomparable(u,v)
    if incomparability.any():
        incomparable_indices = incomparability.nonzero()[0]
        first_pass = comp_spec_feature_vec(put_(u, 
                                                incomparable_indices,
                                                -99),
                                           put_(v, 
                                                incomparable_indices,
                                                -99)).astype(np.float16)
        second_pass = put_(first_pass, incomparable_indices, np.NaN)
        return second_pass
#         raise Exception('u and v must be *completely* comparable')
    return comp_spec_feature_vec(u,v)

# incomparable(np.array([0,1,1]),
#              np.array([0,1,1]))

compare_spec(np.array([0,1,1]),
             np.array([0,1,1]))

compare_spec(np.array([0,1,0]),
             np.array([0,1,1]))

compare_spec(np.array([-1,1,0]),
             np.array([0,1,1]))

incomparable(np.array([0,1,-1]),
             np.array([0,1,1]))

compare_spec(np.array([0,1,-1]),
             np.array([0,1,1]))

print('-'*80)

def compare_specification(u, v):
    '''
    Given two pfvs u, v, returns 
        +1   iff ⟦v⟧ ⊂ ⟦u⟧
         0   iff ⟦u⟧ == ⟦v⟧
        -1   iff ⟦u⟧ ⊂ ⟦v⟧
        NaN  iff ⟦u⟧ and ⟦v⟧ are incomparable
    where
        ⟦⸱⟧ 
    is defined with respect to the set of *all 
    logically possible* objects given the feature system.
    
    I.e. at the level of a single feature value f
         f ⊆ f, ∀f ∈ {-1,0,+1}
        +1 ⊂ 0
        -1 ⊂ 0
        (-1, +1 are incomparable)
    '''
#     if type(u) == np.ndarray and type(v) == np.ndarray:
#         backend = np
#     elif type(u) == torch.Tensor and type(v) == torch.Tensor:
#         backend = torch
#     else:
#         raise Exception('u,v must both either be of type np.ndarray or torch.Tensor')
    if incomparable(u,v).any():
        return np.NaN
    
    elementwise_comp = compare_spec(u,v)
    if (elementwise_comp == 0).all():
        return 0
    elif ((elementwise_comp == 0) | (elementwise_comp == 1)).all():
        return 1
    elif ((elementwise_comp == 0) | (elementwise_comp == -1)).all():
        return -1
    else:
        return np.NaN

compare_specification(np.array([0,1,1]),
                      np.array([0,1,1]))
    
compare_specification(np.array([0,1,0]),
                      np.array([0,1,1]))

compare_specification(np.array([0,1,1]), 
                      np.array([0,1,0]))

#incomparable pairs
compare_specification(np.array([-1,1,0]),
                      np.array([0,1,1]))

compare_specification(np.array([0,1,-1]),
                      np.array([0,1,1]))

array([0, 0, 0])

array([0, 0, 1])

array([-1,  0,  1])

array([False, False,  True])

array([ 0.,  0., nan], dtype=float16)

...

0

1

-1

nan

nan

In [194]:
incomparable(np.array([0,1,1]), np.array([0,1,1]))
u,v = np.array([0,1,1]), np.array([0,1,1])
np.equal(u, -1) & np.equal(v, 1)
np.equal(v, -1) & np.equal(u, 1)

array([False, False, False])

array([False, False, False])

array([False, False, False])

## Union

The union of two partial feature vectors $u,v$ that agree should result in a partial feature vector that has every specified value in $u$, every specified value in $v$, and no other specified values.

In general, the result is at least as specified as either $u$ or $v$: when $u=v$, then $u \cup v = u = v$ and $u \cup v$ is no more specified, but otherwise $u \cup v$ will be strictly more specified than either $u$ or $v$.

In [195]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs

def cup(x,y):
    '''
    Formula:
    x or y, where 1 = T, -1 = T, 0 = F
    
    Algebra:
    0 is the identity ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    (-1 and +1 are mutual inverses, but this case shouldn't occur when agree(x,y) holds)
    
    Pattern:
    x ∪ x = x
    
    0 ∪ y = y
    x ∪ 0 = x
    
    _ ∪ _ = 0  \\ <- shouldn't occur in two pfvs that agree
    '''
    if x == 0:  #if x is unspecified, return y
        return y
    elif y == 0: #if y is unspecified, return x
        return x
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

for x,y in XYs:
    ((x,y), cup(x,y), np.sign(x+y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1, -1)

((-1, 0), -1, -1)

((-1, 1), 0, 0)

((0, -1), -1, -1)

((0, 0), 0, 0)

((0, 1), 1, 1)

((1, -1), 0, 0)

((1, 0), 1, 1)

((1, 1), 1, 1)

In [196]:
def union(u=None, v=None, M=None):
    if u is not None and v is not None:
        if CAREFUL:
            assert agree_(u,v)
        return np.sign(u + v)
    elif M is not None:
        return np.sign(np.sum(M, axis=0)).astype(np.int8)
    else:
        raise Exception('Either provide two operands u,v or a stack of vectors M')

def union_t(u=None, v=None, M=None):
    if u is not None and v is not None:
    #     if CAREFUL:
    #         assert agree_(u,v)
        return torch.sign(u + v).type(torch.int8)
    elif M is not None:
        return torch.sign(torch.sum(M, dim=0)).type(torch.int8)
    else:
        raise Exception('Either provide two operands u,v or a stack of vectors M')


In [197]:
np.array([0,1,1]) + np.array([0,1,1])
np.sum(np.stack([np.array([0,1,1]), np.array([0,1,1])]), axis=0)

array([0, 2, 2])

array([0, 2, 2])

In [198]:
print('u ∪ v where v == u:')
print('\t  u ∪ v = u = v')
print('\t  specification stays the same')
compare_spec(np.array([0,1,1]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,1]),
                      np.array([0,1,1]))
union(np.array([0,1,1]),
      np.array([0,1,1]))
print('-'*80)

print('u ∪ v where v ⊂ u:')
print('\t  u ∪ v = v')
print('\t  specification is as specific as the *most* specified of {u,v}')
compare_spec(np.array([0,1,0]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,0]),
                      np.array([0,1,1]))
union(np.array([0,1,0]),
      np.array([0,1,1]))
print('-'*80)

print('u ∪ v where u and v agree, but are incomparable:')
print('\t  u ∪ v has all the specified features of both u and v')
print('\t  specification is greater than either of {u,v}')
compare_spec(np.array([-1,1,0]),
             np.array([0,1,1]))
compare_specification(np.array([-1,1,0]),
                      np.array([0,1,1]))
union(np.array([-1,1,0]),
      np.array([0,1,1]))
print('-'*80)

print('u ∪ v where u and v disagree (+ are incomparable) only on one feature (viz. last one):')
print('\t  u ∪ v has all the agreeing specified features of both u and v')
print('\t  specification is less than either of {u,v}')
compare_spec(np.array([0,1,-1]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,-1]),
                      np.array([0,1,1]))
union(np.array([0,1,-1]),
      np.array([0,1,1]))
print('-'*80)

u ∪ v where v == u:
	  u ∪ v = u = v
	  specification stays the same


array([0, 0, 0])

0

array([0, 1, 1])

--------------------------------------------------------------------------------
u ∪ v where v ⊂ u:
	  u ∪ v = v
	  specification is as specific as the *most* specified of {u,v}


...

--------------------------------------------------------------------------------
u ∪ v where u and v disagree (+ are incomparable) only on one feature (viz. last one):
	  u ∪ v has all the agreeing specified features of both u and v
	  specification is less than either of {u,v}


array([ 0.,  0., nan], dtype=float16)

nan

array([0, 1, 0])

--------------------------------------------------------------------------------


## Intersection

The intersection of two partial feature vectors $u,v$ should result in a partial feature vector that has every specified value that is specified in both $u$ and $v$ and where $u$ and $v$ agree, and no other specified values.

In general, the result is no more specified than either $u$ or $v$: when $u=v$, $u \cap v = u = v$ and $u \cap v$ is no less specified, but otherwise $u \cap v$ will be strictly less specified than either $u$ or $v$.

In [199]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs 
    
def cap(x,y):
    '''
    Algebra:
    0 is the annihilating element ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    -1 and +1 annihilate each other
    
    Pattern:
    x ∩ x = x
    
    0 ∩ _ = 0
    _ ∩ 0 = 0
    
    _ ∩ _ = 0
    '''
    if x == 0: #if x is unspecified, return 0
        return 0
    elif y == 0: #if y is unspecified, return 0
        return 0
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

def foo(x,y):
    return np.sign( (x == y) * (x + y) )

# def bar(x,y):
#     return (x == y) * (x + y) * 0.5

# def baz(x,y):
#     return (x == y) * int((x + y) / 2)

for x,y in XYs:
#     ((x,y), cap(x,y))
#     ((x,y), cap(x,y), foo(x,y), bar(x,y), baz(x,y))
    ((x,y), cap(x,y), foo(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1, -1)

((-1, 0), 0, 0)

((-1, 1), 0, 0)

((0, -1), 0, 0)

((0, 0), 0, 0)

((0, 1), 0, 0)

((1, -1), 0, 0)

((1, 0), 0, 0)

((1, 1), 1, 1)

In [200]:
def intersection(u=None, v=None, M=None):
    if u is not None and v is not None:
        return np.sign(  np.equal(u, v) * (u + v) )
    elif M is not None:
        return np.sign( np.equal.reduce(M, axis=0) * np.sum(M, axis=0) ).astype(np.int8)
#         return np.sign( np.equal.accumulate(M, axis=0)[-1] * np.sum(M, axis=0) ).astype(np.int8)
#         u = M[0]
#         return np.sign( np.equal(u, M).prod(axis=0) * np.sum(M, axis=0) ).astype(np.int8)
    else:
        raise Exception('Either provide two operands u,v or a stack of vectors M')

def intersection_t(u=None, v=None, M=None):
    if u is not None and v is not None:
        return torch.sign( torch.eq(u,v).type(torch.int8) * (u + v) )
    elif M is not None:
        u = M[0]
        return torch.sign( torch.eq(u, M).prod(dim=0, dtype=torch.int64) * torch.sum(M, dim=0) ).type(torch.int8)
    else:
        raise Exception('Either provide two operands u,v or a stack of vectors M')

In [201]:
random_pair_stack_a.shape
# random_pair_stack_a, random_pair_stack_b

(100000, 23)

In [202]:
for i, each in tqdm(enumerate(random_pair_stack_a)):
    u = each
    v = random_pair_stack_b[i]
    M = np.stack([u,v])
#     intersection(u, v).dtype
#     intersection(M=M).dtype
    assert np.array_equal(intersection(u, v), intersection(M=M))
#     break
    u = random_pair_stack_a_t[i]
    v = random_pair_stack_b_t[i]
    M = torch.stack([u,v])
#     u.dtype
#     v.dtype
#     M.dtype
#     torch.eq(u,v).dtype
#     (u+v).dtype
#     (torch.eq(u,v) * (u+v)).dtype
#     intersection_t(u, v).dtype
# #     intersection_t(M=M).dtype
#     u = M[0]
#     torch.eq(u, M).dtype
#     torch.eq(u, M).prod(dim=0 ,dtype=torch.int8).dtype
#     torch.sum(M, dim=0).dtype
#     (torch.eq(u, M).prod(dim=0 ,dtype=torch.int8) * torch.sum(M, dim=0)).dtype
    assert torch.equal(intersection_t(u, v), intersection_t(M=M))
    
del u
del v
del M



0it [00:00, ?it/s]

880it [00:00, 8796.95it/s]

1794it [00:00, 8896.83it/s]

2716it [00:00, 8991.28it/s]

3636it [00:00, 9050.83it/s]

4554it [00:00, 9087.49it/s]

5470it [00:00, 9108.16it/s]

6387it [00:00, 9126.38it/s]

7303it [00:00, 9136.36it/s]

8223it [00:00, 9154.94it/s]

9140it [00:01, 9159.34it/s]

10062it [00:01, 9174.99it/s]

10980it [00:01, 9176.39it/s]

11888it [00:01, 9146.34it/s]

12809it [00:01, 9162.95it/s]

13731it [00:01, 9178.84it/s]

14657it [00:01, 9201.31it/s]

15577it [00:01, 9198.04it/s]

16495it [00:01, 9186.72it/s]

17413it [00:01, 9178.30it/s]

18330it [00:02, 9172.19it/s]

19254it [00:02, 9191.31it/s]

20179it [00:02, 9208.20it/s]

21105it [00:02, 9220.93it/s]

22031it [00:02, 9232.42it/s]

22955it [00:02, 9193.85it/s]

23881it [00:02, 9211.34it/s]

24803it [00:02, 9193.83it/s]

25723it [00:02, 9184.18it/s]

26642it [00:02, 9179.15it/s]

27567it [00:03, 9197.53it/s]

28487it [00:03, 9176.01it/s]

29409it [00:03, 9187.20it/s]

30332it [00:03, 9199.77it/s]


In [203]:
A = torch.tensor([[0, 1,1], [0,1,1]]); A
A[0]
B = torch.tensor([[-1,1,0], [0,1,1]]); B

torch.eq(A[0], A)
# torch.eq(A[0].unsqueeze(0), A)
torch.eq(A[0], B)
# torch.eq(A[0].unsqueeze(0), B)
torch.eq(A[0], B).prod(dim=0)

tensor([[0, 1, 1],
        [0, 1, 1]])

tensor([0, 1, 1])

tensor([[-1,  1,  0],
        [ 0,  1,  1]])

tensor([[True, True, True],
        [True, True, True]])

tensor([[False,  True, False],
        [ True,  True,  True]])

tensor([0, 1, 0])

In [204]:
print('u ∩ v where v == u:')
print('\t  u ∩ v = u = v')
print('\t  specification stays the same')
compare_spec(np.array([0,1,1]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,1]),
                      np.array([0,1,1]))
intersection(np.array([0,1,1]),
             np.array([0,1,1]))
print('-'*80)

print('u ∩ v where v ⊂ u:')
print('\t  u ∩ v = u')
print('\t  specification is as specific as the *least* specified of {u,v}')
compare_spec(np.array([0,1,0]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,0]),
                      np.array([0,1,1]))
intersection(np.array([0,1,0]),
             np.array([0,1,1]))
print('-'*80)

print('u ∩ v where u and v agree, but are incomparable:')
print('\t  u ∩ v has all the specifications common across u and v')
print('\t  specification is no greater than either of {u,v}, and in general less')
compare_spec(np.array([-1,1,0]),
             np.array([0,1,1]))
compare_specification(np.array([-1,1,0]),
                      np.array([0,1,1]))
intersection(np.array([-1,1,0]),
             np.array([0,1,1]))
print('-'*80)

print('u ∩ v where u and v disagree (+ are incomparable) only on one feature (viz. last one):')
print('\t  u ∩ v has only the specifications common across both u and v')
print('\t  specification is less than either of {u,v}')
compare_spec(np.array([0,1,-1]),
             np.array([0,1,1]))
compare_specification(np.array([0,1,-1]),
                      np.array([0,1,1]))
intersection(np.array([0,1,-1]),
             np.array([0,1,1]))
print('-'*80)

u ∩ v where v == u:
	  u ∩ v = u = v
	  specification stays the same


array([0, 0, 0])

0

array([0, 1, 1])

--------------------------------------------------------------------------------
u ∩ v where v ⊂ u:
	  u ∩ v = u
	  specification is as specific as the *least* specified of {u,v}


...

--------------------------------------------------------------------------------
u ∩ v where u and v disagree (+ are incomparable) only on one feature (viz. last one):
	  u ∩ v has only the specifications common across both u and v
	  specification is less than either of {u,v}


array([ 0.,  0., nan], dtype=float16)

nan

array([0, 1, 0])

--------------------------------------------------------------------------------


## Extension

In [205]:
def getIndex(o, O):
    matches = [i for i,v in enumerate(O) if np.array_equal(v,o)]
    if len(matches) == 0:
        return -1
    if CAREFUL:
        assert len(matches) == 1
    return matches[0]

In [260]:
def makeExtensionVector(positive_Indices, O):
#     return np.array([1 if i in positive_Indices else 0 for i in np.arange(O.shape[0])], dtype=myint)
    return put_(np.zeros(O.shape[0], dtype=myint), positive_Indices, 1, copy_arg=False)

In [207]:
def FVsToExtensionVector(stack_of_fvs, O):
    '''
    Given a (b,m) stack of length-m feature vectors s.t.
    each feature vector corresponds *exactly* to a single object
    in O, this generates the corresponding extension vector.
    '''
#     b = stack_of_fvs.shape[0]
#     n = 
    assert stack_of_fvs.shape[0] < O.shape[0], 'since stack of fvs must be of unique fvs, the size of the stack must be less than the number of objects in O'
    indices = np.array([getIndex(o, O) for o in stack_of_fvs])
    assert indices.shape[0] == np.unique(indices).shape[0], 'stack must be of *unique* feature vectors'
    assert indices.shape[0] == stack_of_fvs.shape[0], 'every object in the stack must be in O'
    return makeExtensionVector(indices, O)

In [208]:
def extension(v, O, asIndexVector=True):
    '''
    The extension of a partial feature vector v is the set of object vectors
    (= fully specified, or 'total' feature vectors) that 'agree' with it.
    '''
    matches = tuple([o for o in O if agree_(v,o)])
#     matches = tuple([o for o in objects if agree(v,o).all()])
#     matches = np.array([1.0 if np.linalg.norm(agree(v,o), 1) == num_features else 0.0 for o in objects])
    if asIndexVector:
        return makeExtensionVector([getIndex(o, O) for o in matches], O)
    return matches

In [209]:
def ramp(M):
    return np.heaviside(M-1, 1).astype(myint)

def primed(p):
    mag_p = np.sum(np.abs(p))
    return p / mag_p

def extension_alt3(s, O):
    if np.array_equal(s, np.zeros(s.shape)):
        return np.ones((l,), dtype=myint)
    p = s
#     mag_p = np.sum(np.abs(p))
#     p_prime = p / mag_p
    return ramp( np.dot(O, primed(p)) )

def heaviside_t(M):
    return M >= 0

def ramp_t(M):
    return heaviside_t(M-1).type(torch.int8)

def primed_t(p):
    if p.device.type == 'cuda':
        mag_p = torch.sum(torch.abs(p.type(torch.int32)))
    else:
        mag_p = torch.sum(torch.abs(p))
    return p / mag_p

def extension_alt3_t(s, O):
    if torch.equal(s, torch.zeros(s.shape, dtype=torch.int8, device=s.device)):
        return torch.ones((l,), dtype=torch.int8)
    p = s
    #FIXME broadcasting is different in pytorch compared to numpy
    return ramp_t( torch.dot(O, primed_t(p)) )

In [210]:
def extension_(pfv, O):
    return agree_mat(pfv, O)

In [271]:
def extensions_(pfvs, O):
    return agree_m(pfvs[:, None, :], O[None, :, :], axis=2).astype(myint)

In [211]:
O_t = torch.from_numpy(O.astype(np.int32)).type(torch.int8)

In [212]:
def extension_t(pfv_t, O_t):
    return agree_mat_t(pfv_t, O_t)

In [213]:
def extensions_t(pfvs_t, O_t):
    return agree_mt(pfvs_t.unsqueeze(1), O_t[None, :, :], dim=2).type(torch.int8)

In [214]:
num_test_pairs = int(1e5)
random_vectors = [make_random_pfv() for each in tqdm(range(num_test_pairs))]
# random_vectors = [choice(objects) for each in tqdm(range(num_test_pairs))]
random_vectors_t = [torch.from_numpy(v.astype(np.int32)).type(torch.int8) for v in random_vectors]
len(random_vectors)



  0%|          | 0/100000 [00:00<?, ?it/s]

 23%|██▎       | 23473/100000 [00:00<00:00, 234726.87it/s]

 48%|████▊     | 47977/100000 [00:00<00:00, 237727.57it/s]

 72%|███████▏  | 72262/100000 [00:00<00:00, 239240.35it/s]

 96%|█████████▋| 96428/100000 [00:00<00:00, 239960.70it/s]

100%|██████████| 100000/100000 [00:00<00:00, 240014.51it/s]

100000

In [215]:
random_vectors_w_nonempty_ext = [v 
                                 for v in tqdm(random_vectors) 
                                 if extension_(v, O).sum() != 0]
len(random_vectors_w_nonempty_ext)



  0%|          | 0/100000 [00:00<?, ?it/s]

  3%|▎         | 3463/100000 [00:00<00:02, 34629.37it/s]

  7%|▋         | 6892/100000 [00:00<00:02, 34526.72it/s]

 10%|█         | 10321/100000 [00:00<00:02, 34455.32it/s]

 14%|█▎        | 13745/100000 [00:00<00:02, 34389.73it/s]

 17%|█▋        | 17144/100000 [00:00<00:02, 34266.94it/s]

 21%|██        | 20570/100000 [00:00<00:02, 34264.50it/s]

 24%|██▍       | 23965/100000 [00:00<00:02, 34167.16it/s]

 27%|██▋       | 27437/100000 [00:00<00:02, 34330.05it/s]

 31%|███       | 30840/100000 [00:00<00:02, 34238.41it/s]

 34%|███▍      | 34254/100000 [00:01<00:01, 34207.83it/s]

 38%|███▊      | 37667/100000 [00:01<00:01, 34182.69it/s]

 41%|████      | 41084/100000 [00:01<00:01, 34176.28it/s]

 45%|████▍     | 44513/100000 [00:01<00:01, 34209.74it/s]

 48%|████▊     | 47975/100000 [00:01<00:01, 34330.63it/s]

 51%|█████▏    | 51431/100000 [00:01<00:01, 34397.10it/s]

 55%|█████▍    | 54892/100000 [00:01<00:01, 34460.46it/s]

 58%|█████▊ 

4914

In [224]:
rv = choice(random_vectors_w_nonempty_ext); rv
extension_t(torch.from_numpy(rv.astype(np.int16)).type(torch.int8), O_t)

array([-1,  0,  0, -1,  1, -1, -1, -1,  0,  1,  0, -1, -1,  0, -1,  1, -1,
        0, -1, -1, -1,  1,  1], dtype=int8)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cpu',
       dtype=torch.uint8)

In [ ]:
# %%timeit

#slow AF - 12m on brh
# [extension(v, O) for v in tqdm(random_vectors)]

In [225]:
%%timeit

lmap(lambda v: extension_alt3(v, O), random_vectors)

2.19 s ± 69.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [226]:
%%timeit

lmap(lambda v: extension_(v,O), random_vectors)

2.32 s ± 7.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [227]:
%%timeit

lmap(lambda v: extension_t(v,O_t), random_vectors_t)

5.89 s ± 35.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [228]:
if torch.cuda.is_available():
    O_tc = O_t.cuda()

In [229]:
rv_t = choice(random_vectors_t)
rv_t

tensor([ 1,  0,  0,  1, -1, -1,  0,  0,  0,  1, -1,  0, -1,  1, -1, -1,  0, -1,
        -1, -1,  1,  0,  0], device='cpu', dtype=torch.int8)

In [230]:
if torch.cuda.is_available():
    agree_mat_t(rv_t.cuda(), O_tc)
    agree_mt(rv_t.cuda(), O_tc, dim=1)
    agree_mt(rv_t.cuda(), O_tc, dim=1).shape

    # agree_mt(random_pair_stack_a_tc, O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

torch.Size([91])

In [231]:
if torch.cuda.is_available():    
    rv_t.shape
    O_tc.shape
    ' '
    random_pair_stack_a_tc.shape
    O_tc.shape
    # random_pair_stack_a_tc

torch.Size([23])

torch.Size([91, 23])

' '

torch.Size([100000, 23])

torch.Size([91, 23])

In [232]:
if torch.cuda.is_available():    
    agree_mt(rv_t.cuda(), O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [233]:
if torch.cuda.is_available():    
    agree_mt(rv_t.cuda()[None, :], O_tc, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [234]:
if torch.cuda.is_available():    
    random_pair_stack_a_tc.shape
    O_tc.shape

    (random_pair_stack_a_tc.unsqueeze(1) * O_tc[None, :, :]).shape

torch.Size([100000, 23])

torch.Size([91, 23])

torch.Size([100000, 91, 23])

In [235]:
if torch.cuda.is_available():    
    agree_mt(random_pair_stack_a_tc.unsqueeze(1), O_tc[None, :, :], dim=2).shape

torch.Size([100000, 91])

In [236]:
# def agree_mt(A, B, dim=0):
#     return (~torch.eq(A*B, -1 * torch.ones(A.shape, dtype=A.dtype, device=A.device))).prod(dim=dim)

In [237]:
if torch.cuda.is_available():    
    agree_mt(random_pair_stack_a_tc, random_pair_stack_b_tc, dim=1)

tensor([0, 0, 0,  ..., 0, 0, 0])

In [238]:
if torch.cuda.is_available():
    extension_t(rv_t.cuda(), O_tc)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

In [239]:
# [extension_alt3_t(v.cuda(), O_tc) for v in tqdm(random_vectors_t)]

In [240]:
# %%timeit

if torch.cuda.is_available():
    #takes 90-120s wittgenstein
    [extension_t(v.cuda(), O_tc) for v in tqdm(random_vectors_t)]
    # list(map(lambda v: extension_t(v.cuda(), O_tc), random_vectors_t))
    # lmap(lambda v: extension_t(v.cuda(), O_t.cuda()), random_vectors_t)



  0%|          | 0/100000 [00:00<?, ?it/s]

  1%|          | 703/100000 [00:00<00:14, 7023.58it/s]

  1%|▏         | 1420/100000 [00:00<00:13, 7064.61it/s]

  2%|▏         | 2140/100000 [00:00<00:13, 7104.65it/s]

  3%|▎         | 2855/100000 [00:00<00:13, 7115.55it/s]

  4%|▎         | 3587/100000 [00:00<00:13, 7174.47it/s]

  4%|▍         | 4309/100000 [00:00<00:13, 7186.72it/s]

  5%|▌         | 5040/100000 [00:00<00:13, 7222.57it/s]

  6%|▌         | 5771/100000 [00:00<00:13, 7247.97it/s]

  6%|▋         | 6494/100000 [00:00<00:12, 7242.53it/s]

  7%|▋         | 7190/100000 [00:04<02:34, 602.19it/s] 

  8%|▊         | 7915/100000 [00:04<01:50, 830.68it/s]

  9%|▊         | 8640/100000 [00:04<01:20, 1131.09it/s]

  9%|▉         | 9367/100000 [00:04<00:59, 1514.76it/s]

 10%|█         | 10097/100000 [00:04<00:45, 1987.07it/s]

 11%|█         | 10768/100000 [00:08<03:12, 462.42it/s] 

 11%|█▏        | 11483/100000 [00:09<02:17, 642.77it/s]

 12%|█▏        | 12202/100000 [00:09<01:39

 92%|█████████▏| 92280/100000 [01:17<00:03, 2104.54it/s]

 93%|█████████▎| 92928/100000 [01:21<00:15, 454.23it/s] 

 94%|█████████▎| 93653/100000 [01:21<00:10, 631.92it/s]

 94%|█████████▍| 94387/100000 [01:22<00:06, 870.62it/s]

 95%|█████████▌| 95119/100000 [01:22<00:04, 1183.39it/s]

 96%|█████████▌| 95846/100000 [01:22<00:02, 1580.22it/s]

 97%|█████████▋| 96578/100000 [01:22<00:01, 2066.24it/s]

 97%|█████████▋| 97263/100000 [01:26<00:05, 473.11it/s] 

 98%|█████████▊| 97998/100000 [01:26<00:03, 657.73it/s]

 99%|█████████▊| 98740/100000 [01:26<00:01, 905.22it/s]

 99%|█████████▉| 99482/100000 [01:26<00:00, 1228.92it/s]

100%|██████████| 100000/100000 [01:26<00:00, 1152.99it/s]

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=

In [241]:
if torch.cuda.is_available():
    random_vectors_tc = torch.stack(random_vectors_t).cuda()
    
    len(random_vectors_t)
    random_vectors_t[0].shape
    
    random_vectors_tc.shape

100000

torch.Size([23])

torch.Size([100000, 23])

In [242]:
if torch.cuda.is_available():
    # 100s wittgenstein
    [extension_t(v, O_tc) for v in tqdm(random_vectors_tc)]



  0%|          | 0/100000 [00:00<?, ?it/s]

  1%|          | 764/100000 [00:00<00:13, 7631.31it/s]

  1%|          | 993/100000 [00:01<03:17, 501.07it/s] 

  2%|▏         | 1790/100000 [00:01<02:20, 697.02it/s]

  3%|▎         | 2594/100000 [00:01<01:41, 960.05it/s]

  3%|▎         | 3396/100000 [00:01<01:14, 1304.50it/s]

  4%|▍         | 4189/100000 [00:01<00:55, 1740.83it/s]

  5%|▍         | 4877/100000 [00:06<03:27, 458.15it/s] 

  6%|▌         | 5591/100000 [00:06<02:28, 636.98it/s]

  6%|▋         | 6359/100000 [00:06<01:46, 878.71it/s]

  7%|▋         | 7135/100000 [00:06<01:17, 1197.17it/s]

  8%|▊         | 7912/100000 [00:06<00:57, 1604.21it/s]

  9%|▊         | 8688/100000 [00:06<00:43, 2105.08it/s]

  9%|▉         | 9405/100000 [00:10<03:04, 490.54it/s] 

 10%|█         | 10199/100000 [00:10<02:11, 682.69it/s]

 11%|█         | 10999/100000 [00:10<01:34, 940.85it/s]

 12%|█▏        | 11801/100000 [00:10<01:08, 1279.68it/s]

 13%|█▎        | 12603/100000 [00:11<00:51, 171

100%|██████████| 100000/100000 [01:36<00:00, 1035.08it/s]

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=

In [243]:
# result_l = [extension_t(v, O_tc) for v in tqdm(random_vectors_tc)]

In [273]:
%timeit extensions_(random_pair_stack_a, O)

547 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [244]:
if torch.cuda.is_available():
    %timeit extensions_t(random_pair_stack_a_tc, O_tc)

185 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [245]:
if torch.cuda.is_available():
    extensions_t(random_pair_stack_b_tc, O_tc)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int8)

In [246]:
if torch.cuda.is_available():
    extensions_t(random_pair_stack_b_tc, O_tc)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int8)

In [247]:
if torch.cuda.is_available():
    extensions_t(random_pair_stack_b_t, O_t)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cpu', dtype=torch.int8)

In [248]:
if torch.cuda.is_available():
    extensions_t(random_pair_stack_b_tc, O_tc).shape
    extensions_t(random_pair_stack_b_tc, O_tc).dtype

torch.Size([100000, 91])

torch.int8

In [274]:
interpretation = extension_
interpretations = extensions_
interpretation_t = extension_t
interpretations_t = extensions_t

In [250]:
pfvs_with_nonempty_extension_t

tensor([[-1, -1, -1,  ...,  0,  0,  0],
        [-1, -1, -1,  ...,  0,  0,  0],
        [-1, -1, -1,  ...,  1,  0,  0],
        ...,
        [ 1,  0,  1,  ...,  0,  0,  1],
        [ 1,  0,  1,  ...,  0,  1,  0],
        [ 1,  0,  1,  ...,  0,  1,  1]], device='cpu', dtype=torch.int8)

## Comparison of extensions

In [275]:
def compare_extensions(x,y):
    '''
    Given two *extension vectors* x,y, returns 
        +1  iff x ⊂ y
         0  iff x ⊆ y ∧ y ⊆ x
        -1  iff y ⊂ x
        NaN iff x and y are incomparable
    '''
    x_obj_indices, y_obj_indices = x.nonzero()[0], y.nonzero()[0]
    if np.array_equal(x_obj_indices, y_obj_indices):
        return 0
    elif np.in1d(y_obj_indices, x_obj_indices).all():
        return 1
    elif np.in1d(x_obj_indices, y_obj_indices).all():
        return -1
    else:
        return np.NaN
#     x_obj_indices, y_obj_indices = set(x.nonzero()[0]), set(y.nonzero()[0])
#     if x_obj_indices == y_obj_indices:
#         return 0
#     elif y_obj_indices.issubset(x_obj_indices):
#         return 1
#     elif x_obj_indices.issubset(y_obj_indices):
#         return -1
#     else:
#         return np.NaN

compare_extensions(np.array([0,1,0,1]),
                   np.array([0,1,0,1]))

compare_extensions(np.array([0,1,0,1]),
                   np.array([0,1,0,0]))

compare_extensions(np.array([0,1,0,0]),
                   np.array([0,1,0,1]))

compare_extensions(np.array([0,1,1,0]),
                   np.array([0,1,0,1]))

0

1

-1

nan

## Given an extension, identify the set of compatible pfvs and the set of exacting matching pfvs

In [287]:
def make_random_extension_vector(O, n_objects=None):#, hasPFV=False):
    max_n_objects = O.shape[0]
    if n_objects is None:
        n_objects = np.random.randint(0, max_n_objects+1)

    random_indices = np.arange(0, O.shape[0])
    np.random.shuffle(random_indices)
    selected_indices = random_indices[:n_objects]
    random_extension_vector = put_(np.zeros(O.shape[0], dtype=myint), 
                                   selected_indices,
                                   1,
                                   copy_arg=False)
    return random_extension_vector
        
#     if not hasPFV:
#         random_extension_vector = np.random.randint(0, 2, n_objects, dtype=myint)
#         return random_extension_vector
    
#     random_object_indices = np.random.randint(0, max_n_objects, n_objects)
#     random_objects_as_pfvs = O[random_object_indices]
    
#     return random_objects

def select_random_indices(A, num_indices=1, axis=0):
    random_indices = np.arange(0, A.shape[axis])
    np.random.shuffle(random_indices)
    selected_indices = random_indices[:num_indices]
    return A[selected_indices]

def make_random_observations(O, n_objects):
    return select_random_indices(O, n_objects)
#     random_indices = np.arange(0, O.shape[0])
#     np.random.shuffle(random_indices)
#     selected_indices = random_indices[:n_objects]
#     return O[selected_indices]

def make_random_pfv(withNonEmptyExtension=False, O=None, uniformly=True, geom_param=0.3):
    if uniformly:
        random_pfv = np.random.randint(3, size=m, dtype=myint) - 1
        if type(O) == torch.Tensor:
            random_pfv = torch.from_numpy(random_pfv.astype(np.int32)).type(torch.int8)

        if not withNonEmptyExtension:
            return random_pfv

        while interpretation(random_pfv, O).sum() == 0:
            random_pfv = np.random.randint(3, size=m, dtype=myint) - 1
            if type(O) == torch.Tensor:
                random_pfv = torch.from_numpy(random_pfv.astype(np.int32)).type(torch.int8)
        return random_pfv
    
    num_specified_indices = max(m, np.random.geometric(p=geom_param))
    specified_values = np.random.randint(3, size=num_specified_indices, dtype=myint) - 1
#     specified_indices = #FIXME
    random_pfv = put_(np.zeros(m, dtype=myint), specified_indices, specified_values)
    
    if type(O) == torch.Tensor:
        random_pfv = torch.from_numpy(random_pfv.astype(np.int32)).type(torch.int8)
    
    if not withNonEmptyExtension:
        return random_pfv
    
    while interpretation(random_pfv, O).sum() == 0:
        num_specified_indices = max(m, np.random.geometric(p=geom_param))
        specified_values = np.random.randint(3, size=num_specified_indices, dtype=myint) - 1
#         specified_indices = #FIXME
        random_pfv = put_(np.zeros(m, dtype=myint), specified_indices, specified_values)
        
        if type(O) == torch.Tensor:
            random_pfv = torch.from_numpy(random_pfv.astype(np.int32)).type(torch.int8)
        
    return random_pfv
        

def make_partial_observations_of(v, O, n_observations=1):
    v_x = interpretation_t(v, O)
    total_obs = O[v_x]
    return select_random_indices(total_obs, n_observations)

In [254]:
np.mean(np.random.geometric(p=0.3, size=10000))

3.3035

In [277]:
random_extension = make_random_extension_vector(O)
random_extension
random_extension.shape

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1], dtype=int8)

(91,)

In [278]:
random_observations = make_random_observations(O, 3)
random_observations
random_observations.shape

array([[-1,  1, -1, -1, -1,  1,  1, -1, -1,  0,  0,  0,  1, -1, -1,  1,
        -1, -1, -1, -1, -1,  0,  0],
       [ 1, -1,  1, -1, -1,  1,  1,  1,  0,  0,  0,  0, -1,  1, -1, -1,
         1, -1, -1, -1, -1, -1,  1],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,
         1, -1,  1, -1, -1,  0,  0]], dtype=int8)

(3, 23)

In [283]:
rpfv = make_random_pfv(); rpfv
interpretation(rpfv, O)
interpretation_t(torch.from_numpy(rpfv.astype(np.int32)).type(torch.int8), O_t)

array([ 1, -1,  1,  1,  0,  1, -1, -1, -1,  0, -1,  0,  1,  0,  1, -1,  1,
       -1, -1,  0,  0,  1,  1], dtype=int8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int8)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cpu',
       dtype=torch.uint8)

In [288]:
print('Random pfv with a non-empty extension:')
rpfv = make_random_pfv(True, O); rpfv
rpfv_t = torch.from_numpy(rpfv.astype(np.int32)).type(torch.int8)
print('-'*80)

print('Extension vector:')
rpfv_x = interpretation_t(rpfv_t, O_t); rpfv_x
rpfv_x.nonzero()
rpfv_x.nonzero().shape
print('-'*80)

print('Complete observation of the extension:')
O_t.shape
rpfv_total_obs = O_t[rpfv_x]; rpfv_total_obs
rpfv_total_obs.shape

print('Random partial observation of the extension:')

n_obs = np.random.randint(1, rpfv_total_obs.shape[0]+1)#; n_obs
# obs_indices = np.random.randint(0, rpfv_total_obs.shape[0], n_obs)#; obs_indices
# rpfv_partial_obs = rpfv_total_obs[obs_indices]
# rpfv_partial_obs = rpfv_total_obs[np.random.randint(0, rpfv_total_obs.shape[0], np.random.randint(1, rpfv_total_obs.shape[0]+1))]
rpfv_partial_obs = make_partial_observations_of(rpfv_t, O_t, n_obs)
rpfv_partial_obs
rpfv_partial_obs.shape
rpfv_partial_obs_np = rpfv_partial_obs.type(torch.int16).numpy().astype(np.int8)
rpfv_partial_obs_x = FVsToExtensionVector(rpfv_partial_obs_np, O)
rpfv_partial_obs_x

Random pfv with a non-empty extension:


array([ 0,  0,  1, -1,  1,  0,  0,  0,  1,  0,  0,  1,  1,  1,  1,  1,  0,
        0,  0,  0, -1,  1,  0], dtype=int8)

--------------------------------------------------------------------------------
Extension vector:


tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cpu',
       dtype=torch.uint8)

tensor([[ 8],
        [ 9],
        [18],
        [28],
        [36],
        [47],
        [48],
        [49],
        [76],
        [78],
        [81]], device='cpu')

...

Random partial observation of the extension:


/opt/conda/conda-bld/pytorch_1565272271120/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


tensor([[-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0, -1,  1,
         -1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,  1, -1,
          1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0, -1,  1,
         -1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1,  1,  0,  1, -1,  0,  0,  0,  0,  0,  1, -1,
         -1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0,  1, -1,
          1, -1, -1,  0,  0],
        [-1, -1,  1, -1,  1, -1,  1,  1,  0,  1, -1,  0,  0,  0,  0,  0,  1, -1,
         -1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0,  1, -1,
         -1, -1, -1,  0,  0],
        [-1, -1,  1, -1,  1, -1,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,  1, -1,
         -1, -1, -1,  0,  0],
        [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,  1, -1,
         -1, -1, -1,  0,  0],
 

torch.Size([11, 23])

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int8)

In [290]:
print('Actual generating pfv: ')
rpfv
# rpfv_partial_obs_np = rpfv_partial_obs.type(torch.int16).numpy().astype(np.int8)
print('Observed objects generated from pfv:')
# rpfv_partial_obs_np.shape
rpfv_partial_obs_np
rpfv_partial_obs_x
# rpfv_partial_obs_np.dtype

print('Union of partial obs:')
reduce(union, rpfv_partial_obs_np)
union(M=rpfv_partial_obs_np)
union_t(M=rpfv_partial_obs)
assert np.array_equal(reduce(union, rpfv_partial_obs_np), union(M=rpfv_partial_obs_np))
print('Comparison of union pfv vs. true pfv by specification: ')
compare_specification(union(M=rpfv_partial_obs_np), rpfv.type(torch.int16).numpy())
print('Comparison of union pfv vs. true pfv by extension: ')
compare_extensions(interpretation(union_t(M=rpfv_partial_obs), O_t).numpy(), 
                   rpfv_partial_obs_x)

print('Intersection of partial obs:')
reduce(intersection, rpfv_partial_obs_np)
intersection(M=rpfv_partial_obs_np)
intersection_t(M=rpfv_partial_obs)
assert np.array_equal(reduce(intersection, rpfv_partial_obs_np), intersection(M=rpfv_partial_obs_np))
print('Comparison of intersection pfv vs. true pfv by specification: ')
compare_specification(intersection(M=rpfv_partial_obs_np), rpfv.type(torch.int16).numpy())
print('Comparison of intersection pfv vs. true pfv by extension: ')
compare_extensions(interpretation(intersection_t(M=rpfv_partial_obs), O_t).numpy(), 
                   rpfv_partial_obs_x)

Actual generating pfv: 


array([ 0,  0,  1, -1,  1,  0,  0,  0,  1,  0,  0,  1,  1,  1,  1,  1,  0,
        0,  0,  0, -1,  1,  0], dtype=int8)

Observed objects generated from pfv:


array([[-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0,
        -1,  1, -1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,
         1, -1,  1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,
        -1,  1, -1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1,  1,  0,  1, -1,  0,  0,  0,  0,  0,
         1, -1, -1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0,
         1, -1,  1, -1, -1,  0,  0],
       [-1, -1,  1, -1,  1, -1,  1,  1,  0,  1, -1,  0,  0,  0,  0,  0,
         1, -1, -1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0, -1, -1,  0,  0,  0,  0,  0,
         1, -1, -1, -1, -1,  0,  0],
       [-1, -1,  1, -1,  1, -1,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,
         1, -1, -1, -1, -1,  0,  0],
       [-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,
         1, -1, -1, -1, -1,  0,  0],
       [-1,  1,  1,

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int8)

Union of partial obs:


array([-1,  1,  1, -1,  1, -1,  1, -1,  0,  1,  1,  0,  0,  0,  0,  0,  0,
        0, -1, -1, -1,  0,  0], dtype=int8)

array([-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,  1,
       -1, -1, -1, -1,  0,  0], dtype=int8)

tensor([-1,  1,  1, -1,  1, -1,  1, -1,  0,  1, -1,  0,  0,  0,  0,  0,  1, -1,
        -1, -1, -1,  0,  0], device='cpu', dtype=torch.int8)

AssertionError: 

In [293]:
#3m53s on wittgenstein
P_to_X = {hash(p.tostring()):interpretation(p, O) #FIXME RESUME
          for p in tqdm(pfvs_with_nonempty_extension)}





  0%|          | 0/9115021 [00:00<?, ?it/s]



  0%|          | 3844/9115021 [00:00<03:57, 38432.71it/s]



  0%|          | 7664/9115021 [00:00<03:57, 38361.37it/s]



  0%|          | 11502/9115021 [00:00<03:57, 38364.83it/s]



  0%|          | 15363/9115021 [00:00<03:56, 38435.71it/s]



  0%|          | 19231/9115021 [00:00<03:56, 38507.50it/s]



  0%|          | 23089/9115021 [00:00<03:55, 38527.86it/s]



  0%|          | 26967/9115021 [00:00<03:55, 38600.28it/s]



  0%|          | 30705/9115021 [00:00<03:57, 38223.71it/s]



  0%|          | 34558/9115021 [00:00<03:56, 38314.74it/s]



  0%|          | 38436/9115021 [00:01<03:56, 38451.98it/s]



  0%|          | 42241/9115021 [00:01<03:56, 38330.48it/s]



  1%|          | 46070/9115021 [00:01<03:56, 38316.80it/s]



  1%|          | 49952/9115021 [00:01<03:55, 38465.85it/s]



  1%|          | 53881/9115021 [00:01<03:54, 38707.35it/s]



  1%|          | 57769/9115021 [00:01<03:53, 38758.92it/s]



  1%|          | 6169

  5%|▍         | 424991/9115021 [00:10<03:41, 39165.59it/s]



  5%|▍         | 428908/9115021 [00:11<03:42, 39091.99it/s]



  5%|▍         | 432831/9115021 [00:11<03:41, 39129.65it/s]



  5%|▍         | 436745/9115021 [00:11<03:42, 38918.32it/s]



  5%|▍         | 440638/9115021 [00:11<03:43, 38829.93it/s]



  5%|▍         | 444522/9115021 [00:11<03:43, 38781.57it/s]



  5%|▍         | 448412/9115021 [00:11<03:43, 38814.15it/s]



  5%|▍         | 452294/9115021 [00:11<03:43, 38794.66it/s]



  5%|▌         | 456180/9115021 [00:11<03:43, 38812.34it/s]



  5%|▌         | 460062/9115021 [00:11<03:43, 38737.66it/s]



  5%|▌         | 463940/9115021 [00:11<03:43, 38745.82it/s]



  5%|▌         | 467815/9115021 [00:12<03:43, 38687.16it/s]



  5%|▌         | 471684/9115021 [00:12<03:43, 38633.09it/s]



  5%|▌         | 475548/9115021 [00:12<03:44, 38557.77it/s]



  5%|▌         | 479404/9115021 [00:12<03:44, 38443.00it/s]



  5%|▌         | 483265/9115021 [00:12<03:44, 38491.58i

  9%|▉         | 850159/9115021 [00:21<03:33, 38653.22it/s]



  9%|▉         | 854064/9115021 [00:21<03:33, 38769.27it/s]



  9%|▉         | 857983/9115021 [00:22<03:32, 38894.49it/s]



  9%|▉         | 861873/9115021 [00:22<03:32, 38762.51it/s]



  9%|▉         | 865750/9115021 [00:22<03:33, 38639.61it/s]



 10%|▉         | 869628/9115021 [00:22<03:33, 38679.34it/s]



 10%|▉         | 873533/9115021 [00:22<03:32, 38789.75it/s]



 10%|▉         | 877422/9115021 [00:22<03:32, 38819.67it/s]



 10%|▉         | 881324/9115021 [00:22<03:31, 38876.91it/s]



 10%|▉         | 885232/9115021 [00:22<03:31, 38937.01it/s]



 10%|▉         | 889126/9115021 [00:22<03:31, 38860.07it/s]



 10%|▉         | 893029/9115021 [00:22<03:31, 38908.63it/s]



 10%|▉         | 896923/9115021 [00:23<03:31, 38917.07it/s]



 10%|▉         | 900821/9115021 [00:23<03:30, 38935.83it/s]



 10%|▉         | 904715/9115021 [00:23<03:31, 38873.59it/s]



 10%|▉         | 908603/9115021 [00:23<03:31, 38775.91i

 14%|█▍        | 1269284/9115021 [00:32<03:23, 38534.00it/s]



 14%|█▍        | 1273177/9115021 [00:32<03:22, 38649.74it/s]



 14%|█▍        | 1277043/9115021 [00:32<03:22, 38633.02it/s]



 14%|█▍        | 1280907/9115021 [00:33<03:23, 38544.05it/s]



 14%|█▍        | 1284768/9115021 [00:33<03:23, 38562.28it/s]



 14%|█▍        | 1288668/9115021 [00:33<03:22, 38691.61it/s]



 14%|█▍        | 1292554/9115021 [00:33<03:21, 38740.80it/s]



 14%|█▍        | 1296470/9115021 [00:33<03:21, 38865.10it/s]



 14%|█▍        | 1300379/9115021 [00:33<03:20, 38929.62it/s]



 14%|█▍        | 1304273/9115021 [00:33<03:21, 38756.06it/s]



 14%|█▍        | 1308192/9115021 [00:33<03:20, 38882.53it/s]



 14%|█▍        | 1312081/9115021 [00:33<03:21, 38788.28it/s]



 14%|█▍        | 1315961/9115021 [00:33<03:21, 38705.56it/s]



 14%|█▍        | 1319833/9115021 [00:34<03:21, 38708.13it/s]



 15%|█▍        | 1323704/9115021 [00:34<03:21, 38684.10it/s]



 15%|█▍        | 1327573/9115021 [00:34<

 18%|█▊        | 1684875/9115021 [00:43<03:12, 38687.10it/s]



 19%|█▊        | 1688776/9115021 [00:43<03:11, 38782.09it/s]



 19%|█▊        | 1692655/9115021 [00:43<03:11, 38710.98it/s]



 19%|█▊        | 1696527/9115021 [00:43<03:11, 38645.95it/s]



 19%|█▊        | 1700392/9115021 [00:43<03:11, 38647.15it/s]



 19%|█▊        | 1704264/9115021 [00:44<03:11, 38667.03it/s]



 19%|█▊        | 1708131/9115021 [00:44<03:11, 38665.11it/s]



 19%|█▉        | 1712012/9115021 [00:44<03:11, 38706.04it/s]



 19%|█▉        | 1715883/9115021 [00:44<03:11, 38691.41it/s]



 19%|█▉        | 1719762/9115021 [00:44<03:10, 38719.59it/s]



 19%|█▉        | 1723645/9115021 [00:44<03:10, 38751.00it/s]



 19%|█▉        | 1727533/9115021 [00:44<03:10, 38788.60it/s]



 19%|█▉        | 1731417/9115021 [00:44<03:10, 38803.07it/s]



 19%|█▉        | 1735298/9115021 [00:44<03:10, 38727.03it/s]



 19%|█▉        | 1739174/9115021 [00:44<03:10, 38732.30it/s]



 19%|█▉        | 1743048/9115021 [00:45<

...

 78%|███████▊  | 7149814/9115021 [03:03<00:50, 38839.02it/s]



 78%|███████▊  | 7153732/9115021 [03:03<00:50, 38939.73it/s]



 79%|███████▊  | 7157671/9115021 [03:03<00:50, 39071.68it/s]



 79%|███████▊  | 7161650/9115021 [03:03<00:49, 39282.82it/s]



 79%|███████▊  | 7165633/9115021 [03:03<00:49, 39444.14it/s]



 79%|███████▊  | 7169597/9115021 [03:03<00:49, 39500.45it/s]



 79%|███████▊  | 7173567/9115021 [03:03<00:49, 39558.91it/s]



 79%|███████▊  | 7177524/9115021 [03:03<00:49, 39464.71it/s]



 79%|███████▉  | 7181471/9115021 [03:04<00:49, 39448.42it/s]



 79%|███████▉  | 7185425/9115021 [03:04<00:48, 39475.52it/s]



 79%|███████▉  | 7189384/9115021 [03:04<00:48, 39506.93it/s]



 79%|███████▉  | 7193335/9115021 [03:04<00:48, 39474.29it/s]



 79%|███████▉  | 7197283/9115021 [03:04<00:48, 39433.26it/s]



 79%|███████▉  | 7201227/9115021 [03:04<00:48, 39402.27it/s]



 79%|███████▉  | 7205168/9115021 [03:04<00:48, 39386.21it/s]



 79%|███████▉  | 7209119/9115021 [03:04<

 83%|████████▎ | 7572863/9115021 [03:14<00:39, 39208.97it/s]



 83%|████████▎ | 7576799/9115021 [03:14<00:39, 39252.68it/s]



 83%|████████▎ | 7580725/9115021 [03:14<00:39, 39209.27it/s]



 83%|████████▎ | 7584646/9115021 [03:14<00:39, 39138.59it/s]



 83%|████████▎ | 7588585/9115021 [03:14<00:38, 39212.37it/s]



 83%|████████▎ | 7592542/9115021 [03:14<00:38, 39317.85it/s]



 83%|████████▎ | 7596474/9115021 [03:14<00:38, 39088.39it/s]



 83%|████████▎ | 7600407/9115021 [03:14<00:38, 39159.99it/s]



 83%|████████▎ | 7604352/9115021 [03:14<00:38, 39245.21it/s]



 83%|████████▎ | 7608287/9115021 [03:14<00:38, 39276.21it/s]



 84%|████████▎ | 7612215/9115021 [03:15<00:38, 39272.47it/s]



 84%|████████▎ | 7616151/9115021 [03:15<00:38, 39296.23it/s]



 84%|████████▎ | 7620081/9115021 [03:15<00:38, 39245.08it/s]



 84%|████████▎ | 7624006/9115021 [03:15<00:38, 39182.23it/s]



 84%|████████▎ | 7627925/9115021 [03:15<00:37, 39181.94it/s]



 84%|████████▎ | 7631854/9115021 [03:15<

 88%|████████▊ | 7996743/9115021 [03:24<00:28, 39588.49it/s]



 88%|████████▊ | 8000702/9115021 [03:24<00:28, 39530.91it/s]



 88%|████████▊ | 8004656/9115021 [03:24<00:28, 39473.80it/s]



 88%|████████▊ | 8008667/9115021 [03:25<00:27, 39657.80it/s]



 88%|████████▊ | 8012648/9115021 [03:25<00:27, 39703.27it/s]



 88%|████████▊ | 8016619/9115021 [03:25<00:27, 39657.45it/s]



 88%|████████▊ | 8020594/9115021 [03:25<00:27, 39683.96it/s]



 88%|████████▊ | 8024563/9115021 [03:25<00:27, 39678.18it/s]



 88%|████████▊ | 8028531/9115021 [03:25<00:27, 39637.86it/s]



 88%|████████▊ | 8032495/9115021 [03:25<00:27, 39580.25it/s]



 88%|████████▊ | 8036454/9115021 [03:25<00:27, 39564.74it/s]



 88%|████████▊ | 8040411/9115021 [03:25<00:27, 39529.32it/s]



 88%|████████▊ | 8044364/9115021 [03:25<00:27, 39509.67it/s]



 88%|████████▊ | 8048328/9115021 [03:26<00:26, 39545.78it/s]



 88%|████████▊ | 8052283/9115021 [03:26<00:26, 39515.15it/s]



 88%|████████▊ | 8056257/9115021 [03:26<

 92%|█████████▏| 8420316/9115021 [03:35<00:17, 39408.93it/s]



 92%|█████████▏| 8424270/9115021 [03:35<00:17, 39445.89it/s]



 92%|█████████▏| 8428224/9115021 [03:35<00:17, 39472.59it/s]



 93%|█████████▎| 8432172/9115021 [03:35<00:17, 39440.16it/s]



 93%|█████████▎| 8436117/9115021 [03:35<00:17, 39180.56it/s]



 93%|█████████▎| 8440036/9115021 [03:35<00:17, 39168.75it/s]



 93%|█████████▎| 8443982/9115021 [03:36<00:17, 39254.29it/s]



 93%|█████████▎| 8447908/9115021 [03:36<00:17, 39239.29it/s]



 93%|█████████▎| 8451848/9115021 [03:36<00:16, 39286.76it/s]



 93%|█████████▎| 8455777/9115021 [03:36<00:16, 39251.33it/s]



 93%|█████████▎| 8459748/9115021 [03:36<00:16, 39385.31it/s]



 93%|█████████▎| 8463720/9115021 [03:36<00:16, 39482.22it/s]



 93%|█████████▎| 8467672/9115021 [03:36<00:16, 39491.38it/s]



 93%|█████████▎| 8471637/9115021 [03:36<00:16, 39536.22it/s]



 93%|█████████▎| 8475591/9115021 [03:36<00:16, 39526.60it/s]



 93%|█████████▎| 8479544/9115021 [03:36<

 97%|█████████▋| 8843960/9115021 [03:46<00:06, 39233.53it/s]



 97%|█████████▋| 8847898/9115021 [03:46<00:06, 39276.53it/s]



 97%|█████████▋| 8851826/9115021 [03:46<00:06, 39124.43it/s]



 97%|█████████▋| 8855757/9115021 [03:46<00:06, 39177.08it/s]



 97%|█████████▋| 8859675/9115021 [03:46<00:06, 39168.08it/s]



 97%|█████████▋| 8863610/9115021 [03:46<00:06, 39220.17it/s]



 97%|█████████▋| 8867533/9115021 [03:46<00:06, 39203.45it/s]



 97%|█████████▋| 8871474/9115021 [03:46<00:06, 39263.52it/s]



 97%|█████████▋| 8875401/9115021 [03:46<00:06, 39235.73it/s]



 97%|█████████▋| 8879325/9115021 [03:47<00:06, 39177.28it/s]



 97%|█████████▋| 8883257/9115021 [03:47<00:05, 39217.34it/s]



 98%|█████████▊| 8887197/9115021 [03:47<00:05, 39270.46it/s]



 98%|█████████▊| 8891136/9115021 [03:47<00:05, 39304.15it/s]



 98%|█████████▊| 8895067/9115021 [03:47<00:05, 39263.24it/s]



 98%|█████████▊| 8899046/9115021 [03:47<00:05, 39417.38it/s]



 98%|█████████▊| 8903001/9115021 [03:47<

In [ ]:
#?m on wittgenstein
P_to_X_t = {hash(p.numpy().tostring()):interpretation_t(p, O_t)
            for p in tqdm(pfvs_with_nonempty_extension_t)}





  0%|          | 0/9115021 [00:00<?, ?it/s]



  0%|          | 1372/9115021 [00:00<11:04, 13716.45it/s]



  0%|          | 2773/9115021 [00:00<11:00, 13800.43it/s]



  0%|          | 4183/9115021 [00:00<10:55, 13888.53it/s]



  0%|          | 5598/9115021 [00:00<10:52, 13963.77it/s]



  0%|          | 7020/9115021 [00:00<10:48, 14037.73it/s]



  0%|          | 8441/9115021 [00:00<10:46, 14086.62it/s]



  0%|          | 9865/9115021 [00:00<10:44, 14129.59it/s]



  0%|          | 11268/9115021 [00:00<10:45, 14097.33it/s]



  0%|          | 12690/9115021 [00:00<10:44, 14133.47it/s]



  0%|          | 14119/9115021 [00:01<10:41, 14180.05it/s]



  0%|          | 15545/9115021 [00:01<10:40, 14201.67it/s]



  0%|          | 16976/9115021 [00:01<10:39, 14232.68it/s]



  0%|          | 18386/9115021 [00:01<10:41, 14191.06it/s]



  0%|          | 19814/9115021 [00:01<10:39, 14215.40it/s]



  0%|          | 21236/9115021 [00:01<10:39, 14215.85it/s]



  0%|          | 22654/911

  2%|▏         | 155720/9115021 [00:11<10:40, 13981.69it/s]



  2%|▏         | 157139/9115021 [00:11<10:37, 14041.25it/s]



  2%|▏         | 158544/9115021 [00:11<10:41, 13957.12it/s]



  2%|▏         | 159952/9115021 [00:11<10:39, 13993.01it/s]



  2%|▏         | 161352/9115021 [00:11<10:41, 13962.69it/s]



  2%|▏         | 162750/9115021 [00:11<10:40, 13967.34it/s]



  2%|▏         | 164148/9115021 [00:11<10:40, 13969.35it/s]



  2%|▏         | 165546/9115021 [00:11<10:40, 13970.66it/s]



  2%|▏         | 166944/9115021 [00:12<10:40, 13968.77it/s]



  2%|▏         | 168341/9115021 [00:12<10:41, 13952.71it/s]



  2%|▏         | 169737/9115021 [00:12<10:43, 13905.14it/s]



  2%|▏         | 171134/9115021 [00:12<10:42, 13923.40it/s]



  2%|▏         | 172531/9115021 [00:12<10:41, 13934.91it/s]



  2%|▏         | 173927/9115021 [00:12<10:41, 13941.05it/s]



  2%|▏         | 175322/9115021 [00:12<10:48, 13786.64it/s]



  2%|▏         | 176707/9115021 [00:12<10:47, 13804.44i

  3%|▎         | 307934/9115021 [00:22<10:33, 13894.99it/s]



  3%|▎         | 309324/9115021 [00:22<10:34, 13886.23it/s]



  3%|▎         | 310713/9115021 [00:22<10:36, 13827.09it/s]



  3%|▎         | 312105/9115021 [00:22<10:35, 13852.24it/s]



  3%|▎         | 313498/9115021 [00:22<10:34, 13873.98it/s]



  3%|▎         | 314894/9115021 [00:22<10:33, 13897.35it/s]



  3%|▎         | 316291/9115021 [00:22<10:32, 13917.06it/s]



  3%|▎         | 317685/9115021 [00:22<10:31, 13921.43it/s]



  4%|▎         | 319078/9115021 [00:23<20:29, 7151.72it/s] 



  4%|▎         | 320470/9115021 [00:23<17:30, 8372.13it/s]



  4%|▎         | 321864/9115021 [00:23<15:24, 9511.03it/s]



  4%|▎         | 323234/9115021 [00:23<13:59, 10470.08it/s]



  4%|▎         | 324629/9115021 [00:23<12:56, 11315.22it/s]



  4%|▎         | 326019/9115021 [00:23<12:13, 11981.69it/s]



  4%|▎         | 327390/9115021 [00:23<11:45, 12450.69it/s]



  4%|▎         | 328781/9115021 [00:24<11:23, 12854.89it/

  5%|▌         | 459944/9115021 [00:33<10:21, 13934.39it/s]



  5%|▌         | 461339/9115021 [00:33<10:20, 13938.74it/s]



  5%|▌         | 462749/9115021 [00:33<10:18, 13985.46it/s]



  5%|▌         | 464163/9115021 [00:33<10:16, 14030.84it/s]



  5%|▌         | 465578/9115021 [00:33<10:15, 14063.82it/s]



  5%|▌         | 466991/9115021 [00:33<10:14, 14081.60it/s]



  5%|▌         | 468400/9115021 [00:34<10:16, 14032.79it/s]



  5%|▌         | 469804/9115021 [00:34<10:15, 14034.50it/s]



  5%|▌         | 471208/9115021 [00:34<10:16, 14020.86it/s]



  5%|▌         | 472611/9115021 [00:34<10:18, 13983.02it/s]



  5%|▌         | 474010/9115021 [00:34<10:19, 13958.33it/s]



  5%|▌         | 475407/9115021 [00:34<10:18, 13960.19it/s]



  5%|▌         | 476804/9115021 [00:34<10:21, 13891.32it/s]



  5%|▌         | 478198/9115021 [00:34<10:21, 13903.29it/s]



  5%|▌         | 479591/9115021 [00:34<10:20, 13910.79it/s]



  5%|▌         | 480983/9115021 [00:34<10:21, 13899.17i

  7%|▋         | 613061/9115021 [00:44<10:07, 13996.04it/s]



  7%|▋         | 614463/9115021 [00:44<10:07, 14001.09it/s]



  7%|▋         | 615867/9115021 [00:45<10:06, 14011.79it/s]



  7%|▋         | 617273/9115021 [00:45<10:05, 14023.51it/s]



  7%|▋         | 618676/9115021 [00:45<10:09, 13943.39it/s]



  7%|▋         | 620072/9115021 [00:45<10:09, 13948.16it/s]



  7%|▋         | 621467/9115021 [00:45<10:08, 13947.59it/s]



  7%|▋         | 622862/9115021 [00:45<10:09, 13944.31it/s]



  7%|▋         | 624263/9115021 [00:45<10:08, 13963.43it/s]



  7%|▋         | 625663/9115021 [00:45<10:07, 13973.50it/s]



  7%|▋         | 627061/9115021 [00:45<10:10, 13906.58it/s]



  7%|▋         | 628460/9115021 [00:45<10:09, 13930.01it/s]



  7%|▋         | 629854/9115021 [00:46<10:09, 13929.59it/s]



  7%|▋         | 631254/9115021 [00:46<10:08, 13948.25it/s]



  7%|▋         | 632652/9115021 [00:46<10:07, 13956.29it/s]



  7%|▋         | 634048/9115021 [00:46<10:09, 13915.80i

...

 23%|██▎       | 2106971/9115021 [02:34<09:07, 12800.80it/s]



 23%|██▎       | 2108270/9115021 [02:34<09:05, 12856.22it/s]



 23%|██▎       | 2109558/9115021 [02:34<09:04, 12861.50it/s]



 23%|██▎       | 2110845/9115021 [02:34<09:09, 12737.47it/s]



 23%|██▎       | 2112128/9115021 [02:35<09:08, 12763.11it/s]



 23%|██▎       | 2113405/9115021 [02:35<09:08, 12760.00it/s]



 23%|██▎       | 2114682/9115021 [02:35<09:11, 12689.89it/s]



 23%|██▎       | 2115969/9115021 [02:35<09:09, 12740.85it/s]



 23%|██▎       | 2117251/9115021 [02:35<09:08, 12762.50it/s]



 23%|██▎       | 2118553/9115021 [02:35<09:05, 12836.51it/s]



 23%|██▎       | 2119939/9115021 [02:35<08:52, 13125.67it/s]



 23%|██▎       | 2121326/9115021 [02:35<08:44, 13338.83it/s]



 23%|██▎       | 2122698/9115021 [02:35<08:39, 13450.44it/s]



 23%|██▎       | 2124081/9115021 [02:35<08:35, 13559.90it/s]



 23%|██▎       | 2125456/9115021 [02:36<08:33, 13614.14it/s]



 23%|██▎       | 2126819/9115021 [02:36<

 25%|██▍       | 2254301/9115021 [02:45<08:14, 13868.02it/s]



 25%|██▍       | 2255688/9115021 [02:45<08:16, 13822.77it/s]



 25%|██▍       | 2257082/9115021 [02:45<08:14, 13856.78it/s]



 25%|██▍       | 2258469/9115021 [02:45<08:14, 13860.73it/s]



 25%|██▍       | 2259860/9115021 [02:45<08:14, 13874.19it/s]



 25%|██▍       | 2261248/9115021 [02:45<08:15, 13835.14it/s]



 25%|██▍       | 2262632/9115021 [02:46<08:16, 13801.19it/s]



 25%|██▍       | 2264013/9115021 [02:46<08:18, 13754.27it/s]



 25%|██▍       | 2265401/9115021 [02:46<08:16, 13790.04it/s]



 25%|██▍       | 2266781/9115021 [02:46<08:16, 13791.39it/s]



 25%|██▍       | 2268164/9115021 [02:46<08:16, 13801.96it/s]



 25%|██▍       | 2269546/9115021 [02:46<08:15, 13805.84it/s]



 25%|██▍       | 2270930/9115021 [02:46<08:15, 13813.75it/s]



 25%|██▍       | 2272312/9115021 [02:46<08:16, 13775.85it/s]



 25%|██▍       | 2273694/9115021 [02:46<08:16, 13788.16it/s]



 25%|██▍       | 2275083/9115021 [02:46<

 26%|██▋       | 2401505/9115021 [02:57<08:10, 13689.55it/s]



 26%|██▋       | 2402882/9115021 [02:57<08:09, 13712.04it/s]



 26%|██▋       | 2404254/9115021 [02:57<08:10, 13685.66it/s]



 26%|██▋       | 2405623/9115021 [02:57<08:14, 13580.39it/s]



 26%|██▋       | 2406987/9115021 [02:57<08:13, 13596.09it/s]



 26%|██▋       | 2408356/9115021 [02:57<08:12, 13622.62it/s]



 26%|██▋       | 2409723/9115021 [02:57<08:11, 13633.99it/s]



 26%|██▋       | 2411087/9115021 [02:57<08:13, 13587.94it/s]



 26%|██▋       | 2412453/9115021 [02:57<08:12, 13608.89it/s]



 26%|██▋       | 2413814/9115021 [02:58<08:15, 13533.64it/s]



 26%|██▋       | 2415178/9115021 [02:58<08:13, 13562.94it/s]



 27%|██▋       | 2416553/9115021 [02:58<08:11, 13616.40it/s]



 27%|██▋       | 2417937/9115021 [02:58<08:09, 13679.95it/s]



 27%|██▋       | 2419319/9115021 [02:58<08:08, 13719.33it/s]



 27%|██▋       | 2420693/9115021 [02:58<08:07, 13723.47it/s]



 27%|██▋       | 2422066/9115021 [02:58<

 28%|██▊       | 2548231/9115021 [03:07<07:59, 13687.95it/s]



 28%|██▊       | 2549600/9115021 [03:07<07:59, 13679.27it/s]



 28%|██▊       | 2550971/9115021 [03:08<07:59, 13686.63it/s]



 28%|██▊       | 2552344/9115021 [03:08<07:59, 13696.73it/s]



 28%|██▊       | 2553714/9115021 [03:08<07:59, 13687.39it/s]



 28%|██▊       | 2555083/9115021 [03:08<08:03, 13572.09it/s]



 28%|██▊       | 2556457/9115021 [03:08<08:01, 13621.03it/s]



 28%|██▊       | 2557834/9115021 [03:08<07:59, 13663.71it/s]



 28%|██▊       | 2559201/9115021 [03:08<08:00, 13652.15it/s]



 28%|██▊       | 2560570/9115021 [03:08<07:59, 13660.64it/s]



 28%|██▊       | 2561938/9115021 [03:08<07:59, 13663.35it/s]



 28%|██▊       | 2563305/9115021 [03:08<08:01, 13611.39it/s]



 28%|██▊       | 2564668/9115021 [03:09<08:01, 13610.82it/s]



 28%|██▊       | 2566037/9115021 [03:09<08:00, 13632.90it/s]



 28%|██▊       | 2567405/9115021 [03:09<07:59, 13644.18it/s]



 28%|██▊       | 2568770/9115021 [03:09<

 30%|██▉       | 2695028/9115021 [03:18<07:49, 13676.93it/s]



 30%|██▉       | 2696396/9115021 [03:18<07:51, 13610.36it/s]



 30%|██▉       | 2697768/9115021 [03:18<07:50, 13638.94it/s]



 30%|██▉       | 2699142/9115021 [03:18<07:49, 13666.77it/s]



 30%|██▉       | 2700517/9115021 [03:19<07:48, 13690.36it/s]



 30%|██▉       | 2701890/9115021 [03:19<07:48, 13700.34it/s]



 30%|██▉       | 2703261/9115021 [03:19<07:48, 13692.61it/s]



 30%|██▉       | 2704631/9115021 [03:19<07:49, 13639.80it/s]



 30%|██▉       | 2705997/9115021 [03:19<07:49, 13645.72it/s]



 30%|██▉       | 2707365/9115021 [03:19<07:49, 13654.57it/s]



 30%|██▉       | 2708738/9115021 [03:19<07:48, 13674.51it/s]



 30%|██▉       | 2710107/9115021 [03:19<07:48, 13678.42it/s]



 30%|██▉       | 2711481/9115021 [03:19<07:47, 13695.48it/s]



 30%|██▉       | 2712851/9115021 [03:19<07:49, 13630.13it/s]



 30%|██▉       | 2714224/9115021 [03:20<07:48, 13657.86it/s]



 30%|██▉       | 2715593/9115021 [03:20<

In [ ]:
if torch.cuda.isavailable():
    #?m on wittgenstein
    P_to_X_tc = {hash(p.numpy().tostring()):interpretation_t(p, O_tc)
                 for p in tqdm(pfvs_with_nonempty_extension_tc)}

## Given an extension and an interpretation function , identify the undominated elements

## Entailed pfvs

In [47]:
def specifiable_zero_indices(p, ext_p):
    '''
    Given p and A::(n,m) = ⟦p⟧:
    
    If p_j = 0 and ∀i A_{i,j} = k≠0, then
    p_j is unspecified (i.e. p_j = 0) but 
    can be set to k and yield a co-extensive 
    and more specific pfv p'. (NB: p' entails 
    p.)
    
    This function returns a list of (index, value) pairs
    indicating the set of 0-valued indices of p that can 
    be specified, plus what the common value at that index is.
    
    Correctly specifying any one or any combination
    of the indices in this list of indices will result
    in a more specific vector than p that is coextensive.
    
    From this list, you can construct (or count) all of the
    more specified pfvs that are coextensive with p.
    '''
    A = ext_p
    n = A.shape[0]
    if n == 0:
        return set()
    n_opp = -1.0 * n
#     zeros = np.nonzero(p)[0]
    zero_indices = np.array(tuple(  set(range(len(p))) - set(np.nonzero(p)[0])  ), dtype=myint)
    specifiable_indices = set()
    for j in zero_indices:
        j_col_sum = np.sum(A[:,j])
        if j_col_sum == n:
            specifiable_indices.add((j, 1))
        if j_col_sum == n_opp:
            specifiable_indices.add((j, -1))
    return specifiable_indices

def specify(p, specs):
    '''
    Given a partial feature vector p and a set of
        (index i, non-zero value v)
    pairs where p_i ≠ 0, returns a more specific p'
    where p'_i = v as indicated by spec.
    '''
    p_prime = p.copy()
    for i,v in specs:
        p_prime[i] = v
    return p_prime

def entailed_pfvs(p, O, no_total_fvs = True):
    '''
    Given a partial feature vector p and a set of objects
    (total feature vectors) O, this returns the set of
    partial feature vectors that are strictly more specific
    than p that have the same extension in O.
    '''
    x_p = np.array(extension(p, O, False))
    specifiable_indices = specifiable_zero_indices(p, x_p)
    num_specifiable_indices = len(specifiable_indices)
    specifications = {tuple(combinations(specifiable_indices, r) )
                      for r in range(1, num_specifiable_indices+1)}
    entailed_vectors = np.array([specify(p, spec)
                                 for r_level in specifications 
                                 for spec in r_level], dtype=myint)
    if not no_total_fvs:
        return entailed_vectors
    entailed_pfvs = np.array([v for v in entailed_vectors
                              if len(v.nonzero()[0]) < m])
    return entailed_pfvs

# Generation of $S_i$: all pfvs with exactly $i$ specified values

In [52]:
# from functools import reduce

In [48]:
def grand_union(pfvs):
    return reduce(union, pfvs)

In [49]:
def one_hot_stack(indices):
#     n_values = np.max(indices) + 1
#     n_values = num_features
    n_values = m
    return np.eye(n_values,dtype=myint)[indices] 

In [50]:
def indexChoicesToComponentOptions(index_choices):
    indices = list(index_choices)
    one_hots = one_hot_stack(indices)
#     component_options = tuple([(v, -1 * v) for v in one_hots])
    component_options = ((v, -1 * v) for v in one_hots)
    return component_options

def componentOptionsToChoices(component_options):
#     choice_combinations = tuple(product(*component_options))
    choice_combinations = product(*component_options)
#     return tuple(starmap(union,
#                          choice_combinations))
#     return tuple(map(grand_union,
#                      choice_combinations))
    return map(grand_union, choice_combinations)

def make_Si_naive(i):
    index_choices = combinations(range(m), i)
    componentOptions = (indexChoicesToComponentOptions(c) for c in index_choices)
    componentChoices = (componentOptionsToChoices(o) for o in componentOptions)
#     choices_flattened = reduce(lambda a,b: a + b, componentChoices)
    choices_flattened = tuple(reduce(lambda a,b: chain.from_iterable([a,b]), componentChoices))
    return np.array(choices_flattened)

In [51]:
construct_Si = make_Si_naive

In [52]:
# calculate_Xi = interpretation

#FIXME this can/should be parallelized and memory mapped
def calculate_Xi_naive(Si, O):
    return np.array([interpretation(p, O) for p in Si], dtype=myint)

In [53]:
def heaviside(x):
    return np.array(1 * (x >= 0))

def extension_multi_bool(p_mat,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (M,num_p) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    K_vec = np.sum(abs(p_mat),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat) #shape is (L,num_p)
    return heaviside(E-K_vec[np.newaxis,:])

# calculate_Xi = extension_multi_bool

def calculate_Xi(p_mat, V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (num_p, M) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    p_mat_prime = p_mat.T
    K_vec = np.sum(abs(p_mat_prime),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat_prime) #shape is (L,num_p)
    result = heaviside(E-K_vec[np.newaxis,:]).T
    
#     K_vec_prime = np.sum(abs(p_mat), axis=1)
# #     assert np.array_equal(K_vec_prime, K_vec.T)
#     E_prime = np.dot(p_mat, V.T)
# #     assert np.array_equal(E_prime, E.T)
#     result_prime = heaviside(E_prime-K_vec_prime[:,np.newaxis])
#     assert result_prime.shape == result.shape, '{0} vs. {1}'.format(result_prime.shape, result.shape)
#     assert np.array_equal(result_prime, result.T)    
    return result#_prime


In [54]:
O.shape

(12, 5)

In [55]:
m

5

In [56]:
S3 = construct_Si(3)
S3.shape

(80, 5)

In [57]:
S3.T.shape

(5, 80)

In [58]:
O.shape

(12, 5)

In [59]:
%%timeit

calculate_Xi_naive(S3, O)

1.11 ms ± 5.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [148]:
%%timeit

calculate_Xi(S3, O)

18.2 µs ± 58.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [60]:
calculate_Xi_naive(S3, O).shape
calculate_Xi_naive(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [61]:
calculate_Xi(S3, O).shape
calculate_Xi(S3, O)

(80, 12)

array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

# Generate $\overline{S}_i$, $\overline{X}_i$ by removing vectors with empty extension in $S_i$ + their empty extension in $X_i$

In [58]:
EMPTY = np.zeros((l,), dtype=myint)

In [73]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_naive(Si, Xi):
    return np.array([v for i,v in enumerate(Si) 
#                      if not empty_extension(Xj[i])])
                     if not np.array_equal(EMPTY, Xi[i])])

In [71]:
#FIXME this can/should be parallelized and memory mapped
def make_Si_bar_Xi_bar_naive(Si, Xi):
    non_empty_indices = np.array([i for i,v in enumerate(Si)
                                  if not np.array_equal(EMPTY, Xi[i])])
    Si_bar = np.array([Si[i] for i in non_empty_indices])
    Xi_bar = np.array([Xi[i] for i in non_empty_indices])
    return Si_bar, Xi_bar

In [75]:
def make_Si_bar_Xi_bar_alt(Si, Xi):
#     non_empty_extension_row_indices = np.array([i for i,v in enumerate(Si)
#                                                 if np.sum(Xi[i]) != 0])
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    
    Si_bar = Si[non_empty_extension_row_indices,:]
    Xi_bar = Xi[non_empty_extension_row_indices,:]
    return Si_bar, Xi_bar

In [76]:
construct_Si_bar = make_Si_bar_naive
construct_Si_Xi_bar = make_Si_bar_Xi_bar_alt

In [172]:
S3.shape

(80, 5)

In [110]:
X3 = calculate_Xi_naive(S3, O)

In [159]:
X3.shape

(80, 9)

In [163]:
l, m

(9, 5)

In [162]:
O.shape

(9, 5)

In [169]:
np.sum(X3, axis=1).shape

(80,)

In [175]:
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).shape
np.array([np.sum(X3[i]) for i,v in enumerate(S3)]).nonzero()[0]

(80,)

array([ 0,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 18, 19, 20,
       21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40,
       42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61,
       62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 75, 77, 78])

In [168]:
non_empty_row_indices = np.array([i for i,v in enumerate(S3)
                                  if np.sum(X3[i]) != 0])
non_empty_row_indices.shape

(64,)

In [157]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[0], make_Si_bar_Xi_bar_alt(S3, X3)[0] )

True

In [158]:
np.array_equal( make_Si_bar_Xi_bar_naive(S3, X3)[1], make_Si_bar_Xi_bar_alt(S3, X3)[1] )

True

In [177]:
%%timeit

make_Si_bar_Xi_bar_naive(S3, X3)

286 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [178]:
%%timeit

make_Si_bar_Xi_bar_alt(S3, X3)

9.55 µs ± 55.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Convert $\overline{X}_i$ to a sparse representation

In [62]:
# import sparse

In [78]:
def density(a):
    num_cells = reduce(lambda x,y: x * y, a.shape)
    d = len(np.nonzero(a)[0]) / num_cells
    return d

def sparsity(a):
    return 1 - density(a)

In [79]:
def to_sparse(v):
    return sparse.COO(v)

# Local processing pipeline to generate $\overline{S}_i$ and (dense) $\overline{X}_i$, $\forall i$ 

In [68]:
def construct_Si_bar_Xi_bar(i, O):
    Si = construct_Si(i)
    Xi = calculate_Xi(Si, O)
    Si_bar, Xi_bar = construct_Si_Xi_bar(Si, Xi)
#     Si_bar = construct_Si_bar(Si, Xi)
#     del Si
#     del Xi
    #FIXME you shouldn't have to recalculate the extensions of everything in Si_bar!
#     Xi_bar = calculate_Xi(Si_bar, O)
    return Si_bar, Xi_bar #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [63]:
construct_Si(1)

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [80]:
S1_bar, X1_bar = construct_Si_bar_Xi_bar(1, O)
sparsity(S1_bar)
sparsity(X1_bar)

0.8

0.5

In [81]:
S1_bar

array([[ 1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1]], dtype=int8)

In [82]:
S2_bar, X2_bar = construct_Si_bar_Xi_bar(2, O)
sparsity(S2_bar)
sparsity(X2_bar)

0.6

0.75

In [83]:
S3_bar, X3_bar = construct_Si_bar_Xi_bar(3, O)
sparsity(S3_bar)
sparsity(X3_bar)

0.4

0.8529411764705882

In [84]:
S4_bar, X4_bar = construct_Si_bar_Xi_bar(4, O)
sparsity(S4_bar)
sparsity(X4_bar)

0.19999999999999996

0.8979591836734694

In [85]:
S5_bar, X5_bar = construct_Si_bar_Xi_bar(5, O)
sparsity(S5_bar)
sparsity(X5_bar)

0.0

0.9166666666666666

# Memory-mapping

In [113]:
X5_bar_sparse = to_sparse(X5_bar)

In [114]:
X5_bar_sparse.nbytes

288

In [155]:
def construct_Si_bar_Xi_bar_mmap(i, O):
    s = ''
#     print('i = {0}'.format(i))
    s += 'i = {0}'.format(i) + '\n'
    
    Si_fn = 'S{0}.dat'.format(i)
    n_pfvs = int(binom(m, i) * (2 ** i))
    n_features = m
    Si_shape = (n_pfvs, n_features)
    
#     print('Si_shape: {0}'.format(Si_shape))
    s += 'Si_shape: {0}'.format(Si_shape) + '\n'
    
    Si = np.memmap(Si_fn, dtype=myint, mode='w+', shape = Si_shape)
#     Si = construct_Si(i)
    Si[:] = construct_Si(i)
#     print('Finished writing S{0} to disk as {1} w/ {2} GB'.format(i, Si_fn, Si.nbytes / 1e9))
    s += 'Finished writing S{0} to disk as {1} w/ {2} GB'.format(i, Si_fn, Si.nbytes / 1e9) + '\n'
    
    Xi_fn = 'X{0}.dat'.format(i)
    l = O.shape[0]
    n_objects = l
    Xi_shape = (n_pfvs, n_objects)
    Xi = np.memmap(Xi_fn, dtype=myint, mode='w+', shape = Xi_shape)
    Xi[:] = calculate_Xi(Si, O)
#     print('Finished writing X{0} to disk as {1} w/ {2} GB'.format(i, Xi_fn, Xi.nbytes / 1e9))
#     print(' ')
    s += 'Finished writing X{0} to disk as {1} w/ {2} GB'.format(i, Xi_fn, Xi.nbytes / 1e9) + '\n\n'
    
    Xi_sums = np.sum(Xi, axis=1) #shape is (Si.shape[0],)
    non_empty_extension_row_indices = Xi_sums.nonzero()[0]
    num_ne_pfvs = len(non_empty_extension_row_indices)
#     print('\tFinished identifying {0} pfvs with non-empty extensions.'.format(num_ne_pfvs))
    s += '\tFinished identifying {0} pfvs with non-empty extensions.'.format(num_ne_pfvs) + '\n'
    
    Si_bar_fn = 'S{0}_bar.dat'.format(i)
    Si_bar_shape = (num_ne_pfvs, n_features)
    Si_bar = np.memmap(Si_bar_fn, dtype=myint, mode='w+', shape = Si_bar_shape)
    Si_bar[:] = Si[non_empty_extension_row_indices,:]
#     print('\tFinished writing S{0}_bar to disk as {1} w/ {2} GB'.format(i, Si_bar_fn, Si_bar.nbytes / 1e9))
#     print('\t\tSparsity of S{0}_bar: {1}'.format(i, sparsity(Si_bar)))
#     print('\t\tDeleting S{0} from disk, freeing {1} GB'.format(i, Si.nbytes / 1e9))
    s += '\tFinished writing S{0}_bar to disk as {1} w/ {2} GB'.format(i, Si_bar_fn, Si_bar.nbytes / 1e9) + '\n'
    s += '\t\tSparsity of S{0}_bar: {1}'.format(i, sparsity(Si_bar)) + '\n'
    s += '\t\tDeleting S{0} from disk, freeing {1} GB'.format(i, Si.nbytes / 1e9) + '\n'
    os.remove(Si_fn)
#     print('\t\tS{0} deleted'.format(i))
    s += '\t\tS{0} deleted'.format(i) + '\n'
#     print('{0} GB used for S{1}_bar'.format(Si_bar.nbytes / 1e9, i))
    s += '{0} GB used for S{1}_bar'.format(Si_bar.nbytes / 1e9, i) + '\n'
    
    Xi_bar_fn = 'X{0}_bar.dat'.format(i)
    Xi_bar_shape = (num_ne_pfvs, n_objects)
    Xi_bar = np.memmap(Xi_bar_fn, dtype=myint, mode='w+', shape = Xi_bar_shape)
    Xi_bar[:] = Xi[non_empty_extension_row_indices,:]
    print('\tFinished writing X{0}_bar to disk as {1} w/ {2} GB'.format(i, Xi_bar_fn, Xi_bar.nbytes / 1e9))
    s += '\tFinished writing X{0}_bar to disk as {1} w/ {2} GB'.format(i, Xi_bar_fn, Xi_bar.nbytes / 1e9) + '\n'
#     print('\t\tDeleting X{0} from disk, freeing {1} GB'.format(i, Xi.nbytes / 1e9))
    s += '\t\tDeleting X{0} from disk, freeing {1} GB'.format(i, Xi.nbytes / 1e9) + '\n'
    os.remove(Xi_fn)
#     print('\t\tX{0} deleted'.format(i))
    s += '\t\tX{0} deleted'.format(i) + '\n'
    
#     print('\t\tSparsity of X{0}_bar: {1}'.format(i, sparsity(Xi_bar)))
    s += '\t\tSparsity of X{0}_bar: {1}'.format(i, sparsity(Xi_bar)) + '\n'
#     print('\t\tCreating sparse version of X{0}_bar'.format(i))
    s += '\t\tCreating sparse version of X{0}_bar'.format(i) + '\n'
    Xi_bar_sparse_fn = 'X{0}_bar.sparse'.format(i)
    Xi_bar_sparse = to_sparse(Xi_bar)
#     print('\t\tSaving as {0}'.format(Xi_bar_sparse_fn))
    s += '\t\tSaving as {0}'.format(Xi_bar_sparse_fn) + '\n'
    sparse.save_npz(Xi_bar_sparse_fn, Xi_bar_sparse)
#     print('\t\tSaved, using {0} GB'.format(Xi_bar_sparse.nbytes / 1e9))
    s += '\t\tSaved, using {0} GB'.format(Xi_bar_sparse.nbytes / 1e9) + '\n'
#     print('\t\tDeleting {0}, saving {1} GB'.format(Xi_bar_fn, Xi_bar.nbytes / 1e9))
    s += '\t\tDeleting {0}, saving {1} GB'.format(Xi_bar_fn, Xi_bar.nbytes / 1e9) + '\n'
    
#     print('{0} GB used for sparse X{1}_bar'.format(Xi_bar_sparse.nbytes / 1e9, i))
    s += '{0} GB used for sparse X{1}_bar'.format(Xi_bar_sparse.nbytes / 1e9, i) + '\n'
    
#     print(' ')
    s += '\n'
    print(s)
    return Si_bar, Xi_bar_sparse #these (or at least Xi_bar) should be sparse (and memory mapped) representations

In [127]:
m

5

In [131]:
for i in range(1,m+1):
    print('i = {0}'.format(i))
    construct_Si_bar_Xi_bar_mmap(i, O)
    print(' ')

i = 1
Si_shape: (10, 5)
Finished writing S1 to disk as S1.dat w/ 5e-08 GB
Finished writing X1 to disk as X1.dat w/ 1.2e-07 GB
Finished identifying 10 pfvs with non-empty extensions.
Finished writing S1_bar to disk as S1_bar.dat w/ 5e-08 GB
Sparsity of S1_bar: 0.8
Deleting S1 from disk, freeing 5e-08 GB
S1 deleted
Finished writing X1_bar to disk as X1_bar.dat w/ 1.2e-07 GB
Deleting X1 from disk, freeing 1.2e-07 GB
X1 deleted
Sparsity of X1_bar: 0.5
Creating sparse version of X1_bar
Saving as X1_bar.sparse
Saved, using 1.02e-06 GB
Deleting X1_bar.dat, saving 1.2e-07 GB


(memmap([[ 1,  0,  0,  0,  0],
         [-1,  0,  0,  0,  0],
         [ 0,  1,  0,  0,  0],
         [ 0, -1,  0,  0,  0],
         [ 0,  0,  1,  0,  0],
         [ 0,  0, -1,  0,  0],
         [ 0,  0,  0,  1,  0],
         [ 0,  0,  0, -1,  0],
         [ 0,  0,  0,  0,  1],
         [ 0,  0,  0,  0, -1]], dtype=int8),
 <COO: shape=(10, 12), dtype=int8, nnz=60, fill_value=0>)

 
i = 2
Si_shape: (40, 5)
Finished writing S2 to disk as S2.dat w/ 2e-07 GB
Finished writing X2 to disk as X2.dat w/ 4.8e-07 GB
Finished identifying 40 pfvs with non-empty extensions.
Finished writing S2_bar to disk as S2_bar.dat w/ 2e-07 GB
Sparsity of S2_bar: 0.6
Deleting S2 from disk, freeing 2e-07 GB
S2 deleted
Finished writing X2_bar to disk as X2_bar.dat w/ 4.8e-07 GB
Deleting X2 from disk, freeing 4.8e-07 GB
X2 deleted
Sparsity of X2_bar: 0.75
Creating sparse version of X2_bar
Saving as X2_bar.sparse
Saved, using 2.04e-06 GB
Deleting X2_bar.dat, saving 4.8e-07 GB


(memmap([[ 1,  1,  0,  0,  0],
         [ 1, -1,  0,  0,  0],
         [-1,  1,  0,  0,  0],
         [-1, -1,  0,  0,  0],
         [ 1,  0,  1,  0,  0],
         [ 1,  0, -1,  0,  0],
         [-1,  0,  1,  0,  0],
         [-1,  0, -1,  0,  0],
         [ 1,  0,  0,  1,  0],
         [ 1,  0,  0, -1,  0],
         [-1,  0,  0,  1,  0],
         [-1,  0,  0, -1,  0],
         [ 1,  0,  0,  0,  1],
         [ 1,  0,  0,  0, -1],
         [-1,  0,  0,  0,  1],
         [-1,  0,  0,  0, -1],
         [ 0,  1,  1,  0,  0],
         [ 0,  1, -1,  0,  0],
         [ 0, -1,  1,  0,  0],
         [ 0, -1, -1,  0,  0],
         [ 0,  1,  0,  1,  0],
         [ 0,  1,  0, -1,  0],
         [ 0, -1,  0,  1,  0],
         [ 0, -1,  0, -1,  0],
         [ 0,  1,  0,  0,  1],
         [ 0,  1,  0,  0, -1],
         [ 0, -1,  0,  0,  1],
         [ 0, -1,  0,  0, -1],
         [ 0,  0,  1,  1,  0],
         [ 0,  0,  1, -1,  0],
         [ 0,  0, -1,  1,  0],
         [ 0,  0, -1, -1,  0],
        

 
i = 3
Si_shape: (80, 5)
Finished writing S3 to disk as S3.dat w/ 4e-07 GB
Finished writing X3 to disk as X3.dat w/ 9.6e-07 GB
Finished identifying 68 pfvs with non-empty extensions.
Finished writing S3_bar to disk as S3_bar.dat w/ 3.4e-07 GB
Sparsity of S3_bar: 0.4
Deleting S3 from disk, freeing 4e-07 GB
S3 deleted
Finished writing X3_bar to disk as X3_bar.dat w/ 8.16e-07 GB
Deleting X3 from disk, freeing 9.6e-07 GB
X3 deleted
Sparsity of X3_bar: 0.8529411764705882
Creating sparse version of X3_bar
Saving as X3_bar.sparse
Saved, using 2.04e-06 GB
Deleting X3_bar.dat, saving 8.16e-07 GB


(memmap([[ 1,  1,  1,  0,  0],
         [ 1,  1, -1,  0,  0],
         [ 1, -1,  1,  0,  0],
         [ 1, -1, -1,  0,  0],
         [-1,  1, -1,  0,  0],
         [-1, -1,  1,  0,  0],
         [ 1,  1,  0,  1,  0],
         [ 1,  1,  0, -1,  0],
         [ 1, -1,  0,  1,  0],
         [ 1, -1,  0, -1,  0],
         [-1,  1,  0, -1,  0],
         [-1, -1,  0,  1,  0],
         [ 1,  1,  0,  0,  1],
         [ 1,  1,  0,  0, -1],
         [ 1, -1,  0,  0,  1],
         [ 1, -1,  0,  0, -1],
         [-1,  1,  0,  0,  1],
         [-1, -1,  0,  0, -1],
         [ 1,  0,  1,  1,  0],
         [ 1,  0,  1, -1,  0],
         [ 1,  0, -1,  1,  0],
         [ 1,  0, -1, -1,  0],
         [-1,  0,  1,  1,  0],
         [-1,  0, -1, -1,  0],
         [ 1,  0,  1,  0,  1],
         [ 1,  0,  1,  0, -1],
         [ 1,  0, -1,  0,  1],
         [ 1,  0, -1,  0, -1],
         [-1,  0,  1,  0, -1],
         [-1,  0, -1,  0,  1],
         [ 1,  0,  0,  1,  1],
         [ 1,  0,  0,  1, -1],
        

 
i = 4
Si_shape: (80, 5)
Finished writing S4 to disk as S4.dat w/ 4e-07 GB
Finished writing X4 to disk as X4.dat w/ 9.6e-07 GB
Finished identifying 49 pfvs with non-empty extensions.
Finished writing S4_bar to disk as S4_bar.dat w/ 2.45e-07 GB
Sparsity of S4_bar: 0.19999999999999996
Deleting S4 from disk, freeing 4e-07 GB
S4 deleted
Finished writing X4_bar to disk as X4_bar.dat w/ 5.88e-07 GB
Deleting X4 from disk, freeing 9.6e-07 GB
X4 deleted
Sparsity of X4_bar: 0.8979591836734694
Creating sparse version of X4_bar
Saving as X4_bar.sparse
Saved, using 1.02e-06 GB
Deleting X4_bar.dat, saving 5.88e-07 GB


(memmap([[ 1,  1,  1,  1,  0],
         [ 1,  1,  1, -1,  0],
         [ 1,  1, -1,  1,  0],
         [ 1, -1,  1,  1,  0],
         [ 1, -1,  1, -1,  0],
         [ 1, -1, -1,  1,  0],
         [ 1, -1, -1, -1,  0],
         [-1,  1, -1, -1,  0],
         [-1, -1,  1,  1,  0],
         [ 1,  1,  1,  0,  1],
         [ 1,  1,  1,  0, -1],
         [ 1,  1, -1,  0,  1],
         [ 1,  1, -1,  0, -1],
         [ 1, -1,  1,  0,  1],
         [ 1, -1,  1,  0, -1],
         [ 1, -1, -1,  0,  1],
         [ 1, -1, -1,  0, -1],
         [-1,  1, -1,  0,  1],
         [-1, -1,  1,  0, -1],
         [ 1,  1,  0,  1,  1],
         [ 1,  1,  0,  1, -1],
         [ 1,  1,  0, -1,  1],
         [ 1,  1,  0, -1, -1],
         [ 1, -1,  0,  1,  1],
         [ 1, -1,  0,  1, -1],
         [ 1, -1,  0, -1,  1],
         [ 1, -1,  0, -1, -1],
         [-1,  1,  0, -1,  1],
         [-1, -1,  0,  1, -1],
         [ 1,  0,  1,  1,  1],
         [ 1,  0,  1,  1, -1],
         [ 1,  0,  1, -1,  1],
        

 
i = 5
Si_shape: (32, 5)
Finished writing S5 to disk as S5.dat w/ 1.6e-07 GB
Finished writing X5 to disk as X5.dat w/ 3.84e-07 GB
Finished identifying 12 pfvs with non-empty extensions.
Finished writing S5_bar to disk as S5_bar.dat w/ 6e-08 GB
Sparsity of S5_bar: 0.0
Deleting S5 from disk, freeing 1.6e-07 GB
S5 deleted
Finished writing X5_bar to disk as X5_bar.dat w/ 1.44e-07 GB
Deleting X5 from disk, freeing 3.84e-07 GB
X5 deleted
Sparsity of X5_bar: 0.9166666666666666
Creating sparse version of X5_bar
Saving as X5_bar.sparse
Saved, using 2.04e-07 GB
Deleting X5_bar.dat, saving 1.44e-07 GB


(memmap([[ 1,  1,  1,  1,  1],
         [ 1,  1,  1, -1,  1],
         [ 1,  1,  1, -1, -1],
         [ 1,  1, -1,  1,  1],
         [ 1,  1, -1,  1, -1],
         [ 1, -1,  1,  1, -1],
         [ 1, -1,  1, -1,  1],
         [ 1, -1, -1,  1,  1],
         [ 1, -1, -1,  1, -1],
         [ 1, -1, -1, -1, -1],
         [-1,  1, -1, -1,  1],
         [-1, -1,  1,  1, -1]], dtype=int8),
 <COO: shape=(12, 12), dtype=int8, nnz=12, fill_value=0>)

In [156]:
m = 23

max_num_objects = 2 ** m
max_num_objects
# actual_num_objects = np.random.randint(max_num_objects)
actual_num_objects = 96
actual_num_objects

assert actual_num_objects < max_num_objects

8388608

96

In [157]:
O = makeRandomObjects(actual_num_objects, m, True)
O
O.shape
l = len(O); l

array([[-1,  1, -1, ...,  1,  1, -1],
       [ 1, -1, -1, ..., -1, -1,  1],
       [-1, -1,  1, ...,  1, -1,  1],
       ..., 
       [ 1, -1,  1, ...,  1,  1,  1],
       [ 1,  1, -1, ..., -1,  1, -1],
       [-1, -1,  1, ..., -1,  1,  1]])

(96, 23)

96

In [158]:
J
V

-1

10

In [159]:
# for i in range(1,m+1):
#     print('i = {0}'.format(i))
#     construct_Si_bar_Xi_bar_mmap(i, O)
#     print(' ')

In [ ]:
par(delayed(construct_Si_bar_Xi_bar_mmap)(i, O) for i in range(1, m+1))

i = 2
i = 1
i = 3
i = 4
Si_shape: (46, 23)
Si_shape: (1012, 23)
i = 5
Si_shape: (14168, 23)
Si_shape: (141680, 23)
i = 6
i = 7
Si_shape: (1076768, 23)
Si_shape: (6460608, 23)
i = 8
i = 9
i = 10
Si_shape: (31380096, 23)
Si_shape: (418401280, 23)
Si_shape: (125520384, 23)
i = 11
i = 12
Finished writing S1 to disk as S1.dat w/ 1.058e-06 GB
i = 13
i = 14
Si_shape: (2769055744, 23)
i = 15
Si_shape: (1171523584, 23)
Si_shape: (13388840960, 23)
i = 16
Si_shape: (5538111488, 23)
Si_shape: (9372188672, 23)
i = 17
Si_shape: (16066609152, 23)
Si_shape: (16066609152, 23)
i = 18
i = 21
Si_shape: (8820883456, 23)
i = 19
i = 22
Si_shape: (530579456, 23)
Si_shape: (13231325184, 23)
Si_shape: (4642570240, 23)
Si_shape: (96468992, 23)
i = 20
i = 23
Si_shape: (1857028096, 23)
Si_shape: (8388608, 23)
Finished writing S2 to disk as S2.dat w/ 2.3276e-05 GB
Finished writing X1 to disk as X1.dat w/ 4.416e-06 GB
 
	Finished identifying 46 pfvs with non-empty extensions.
	Finished writing S1_bar to disk as S1_b

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0946s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 out of  23 | elapsed:    0.1s remaining:    1.2s


Finished writing S3 to disk as S3.dat w/ 0.000325864 GB
Finished writing X3 to disk as X3.dat w/ 0.001360128 GB
 
	Finished identifying 14168 pfvs with non-empty extensions.
	Finished writing S3_bar to disk as S3_bar.dat w/ 0.000325864 GB
		Sparsity of S3_bar: 0.8695652173913043
		Deleting S3 from disk, freeing 0.000325864 GB
		S3 deleted
0.000325864 GB used for S0.000325864_bar
	Finished writing X3_bar to disk as X3_bar.dat w/ 0.001360128 GB
		Deleting X3 from disk, freeing 0.001360128 GB
		X3 deleted
		Sparsity of X3_bar: 0.875
		Creating sparse version of X3_bar
		Saving as X3_bar.sparse
		Saved, using 0.002890272 GB
		Deleting X3_bar.dat, saving 0.001360128 GB
0.002890272 GB used for sparse X0.002890272_bar
 
Finished writing S4 to disk as S4.dat w/ 0.00325864 GB
Finished writing X4 to disk as X4.dat w/ 0.01360128 GB
 
	Finished identifying 141387 pfvs with non-empty extensions.
	Finished writing S4_bar to disk as S4_bar.dat w/ 0.003251901 GB
		Sparsity of S4_bar: 0.826086956521739

[Parallel(n_jobs=-1)]: Done   5 out of  23 | elapsed:  4.1min remaining: 14.7min


Finished writing S22 to disk as S22.dat w/ 2.218786816 GB
